<style>
body {
    max-width: 900px;
    margin: 40px auto;
    padding: 0 20px;
    font-family: "Georgia", serif;
    line-height: 1.6;
}
</style>

<div style="text-align: center; padding: 60px 60px">
  <h1 style="font-weight: bold; font-size: 3.1em">
    ENCONTRAR MODELOS QUE CUMPLAN LOS SUPUESTOS
  </h1>
</div>

Dado que en el `02-----Encontrar-los-Mejores-Modelos` se intentó primero encontrar el mejor $\text{AIC}$, y luego se intento verificar los supuestos, y ninguno cumplió los supuestos, se optó por hacerlo al revés y primero encontrar modelos que cumplan los supuestos y luego encontrar el modelo con mejor $\text{AIC}$.

*Observación (1):* Se notó que, para esta serie, los modelos que tenían parte *autorregresiva* y *autorregresiva estacional* no eran estacionarios, coincidia que eran estacionarios si tenían exclusivamente una de las partes, lo mismo para la parte *media móvil*. Tomando en cuenta esta observación, se hicieron las notebooks de `03-01`, ..., `03-04`, donde se varían, por ejemplo, los $p$, manteniendo $P=0$, lo mismo para la parte *media móvil*.

*Observación (2):* También se notó, junto con la notebook anterior, que, para que los residuos del modelo fueran independientes, era preferible tomar polinomios grandes, para que el modelo capturara correctamente la dependencia temporal de la serie. Entonces, no se espera que nuestro modelo cumpla el principio de parsimonia, pero sí que sea un modelo admisible y que cumpla con los supuestos de los errores.

**RESULTADOS:** Esta notebook no llega a ningun resultado, pero nos da bases para empezar con las de `03-01`, ..., `03-04`, con las que se proponen los siguientes tres modelos:

In [1]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from verificacion_de_supuestos import * # Una funcion que hice para verificar si los parametros son estacionarios/invertibles
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import ttest_1samp
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.tools.tools import add_constant
from scipy.stats import jarque_bera
from statsmodels.stats.diagnostic import lilliefors

## **CARGAR LOS DATOS**

In [3]:
import pandas as pd
import numpy as np

data=pd.read_csv('MXN00021035.csv')

pre=data.iloc[:,6]  # Precipitacion, es la columna 5
date=data.iloc[:,5] # Date, es la columna 6
date = date.astype(str).str.replace(r'(\d{4})(\d{2})', r'\1/\2', regex=True)    # La fecha está como 195210 y la pasamos a 1952/10 
date = pd.to_datetime(date, format='%Y/%m')                                     # Lo convertimos en fecha
pre = pd.Series(pre.values, index=date)                                         # Creamos una Serie

# Partir la serie para train y test
pre_total = pre.copy()          # Copia de la serie original

# Todas hasta los ultimos 12 meses
pre = pre_total[:-12]           # Entrenamiento: todos menos los últimos 12 meses

from scipy.stats import zscore

# Estandarizar la serie original
zpre = zscore(pre)

# **MODELADO**

## **1 MODELO**

Este modelo se escogió basado en las autocorrelaciones simples y parciales significativas.

In [4]:
modelo=SARIMAX(zpre,
               order=(1,0,1),
               seasonal_order=(0,1,1,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                                  y   No. Observations:                  660
Model:             SARIMAX(1, 0, 1)x(0, 1, 1, 12)   Log Likelihood                -711.177
Date:                            Sat, 26 Apr 2025   AIC                           1430.355
Time:                                    02:31:38   BIC                           1448.250
Sample:                                         0   HQIC                          1437.297
                                            - 660                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.2786      0.217      1.283      0.199      -0.147       0.704
ma.L1         -0.1149      0.223     -0.515      0.606      -0.552       0.322
ma.S.L12      -0.6862      0.024    -28.283      0.000      -0.734      -0.639
sigma2         0.5196      0.020     26.467      0.000       0.481       0.558
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               228.74
Prob(Q):                              1.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.88   Skew:                             0.51
Prob(H) (two-sided):                  0.36   Kurtosis:                         5.72
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [9]:
acorr_ljungbox(modelo.resid, period=12, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.136052,NaN
2,0.136083,NaN
3,1.431875,NaN
4,6.727948,9.491374e-03
5,26.445328,1.809131e-06
6,41.408615,5.356129e-09
7,41.816104,1.821177e-08
8,42.679221,4.291645e-08
9,44.080115,7.126546e-08
10,47.652562,4.163102e-08


Todos los errores son dependientes, el modelo no captura correctamente la dependencia temporal de la serie, por lo que se añadieron parámetros.

In [82]:
modelo.resid.mean()

11.844449805815001

Además, sus residuos son muy altos.

## **2 MODELO**

Como los p-valores de la prueba de Ljung-Box siempre son muy pequeños (menores a 0.01), se concluyó que el modelo no capturaba correctamente la dependencia temporal de la serie, se pusieron parámetros "intuitivos", es decir, la precipitación depende de los últimos 5 meses, de los últimos 4 años, y hay errores en los últimos dos meses.

In [44]:
modelo=SARIMAX(zpre,
               order=(5,0,2),
               seasonal_order=(4,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                  660
Model:             SARIMAX(5, 0, 2)x(4, 1, [], 12)   Log Likelihood                -692.489
Date:                             Sat, 26 Apr 2025   AIC                           1408.977
Time:                                     03:02:08   BIC                           1462.664
Sample:                                          0   HQIC                          1429.804
                                             - 660                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          1.6517      0.099     16.600      0.000       1.457       1.847
ar.L2         -0.9210      0.110     -8.388      0.000      -1.136      -0.706
ar.L3          0.1293      0.084      1.536      0.124      -0.036       0.294
ar.L4         -0.1955      0.085     -2.308      0.021      -0.361      -0.029
ar.L5          0.1048      0.058      1.805      0.071      -0.009       0.219
ma.L1         -1.5552      0.094    -16.585      0.000      -1.739      -1.371
ma.L2          0.7752      0.077     10.053      0.000       0.624       0.926
ar.S.L12      -0.7547      0.032    -23.428      0.000      -0.818      -0.692
ar.S.L24      -0.5815      0.038    -15.125      0.000      -0.657      -0.506
ar.S.L36      -0.3246      0.040     -8.055      0.000      -0.404      -0.246
ar.S.L48      -0.1726      0.032     -5.383      0.000      -0.235      -0.110
sigma2         0.4898      0.020     24.007      0.000       0.450       0.530
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               219.25
Prob(Q):                              0.95   Prob(JB):                         0.00
Heteroskedasticity (H):               0.86   Skew:                             0.55
Prob(H) (two-sided):                  0.26   Kurtosis:                         5.63
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [12]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.174630,NaN
2,0.185215,NaN
3,0.185767,NaN
4,0.268400,NaN
5,1.648177,NaN
6,3.515678,NaN
7,4.829842,NaN
8,4.938400,NaN
9,5.248623,NaN
10,5.249259,NaN


**TIENE TODAS LAS AUTOCORRELACIONES POSITIVAS! para un $\alpha = 0.01$**

In [13]:
parsimonia(modelo) 

Hay coeficientes no significativos, no se cumple el principio de parsimonia


In [14]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p1': -1.6516504044542035, 'p2': 0.9209574967760907, 'p3': -0.12925980281218555, 'p4': 0.19545911419107637, 'p5': -0.10478999490571805, 'p12': 0.7547204387045899, 'p24': 0.581511877984189, 'p36': 0.3245778402640002, 'p48': 0.17262836604891985}

El grado del polinomio es: 48

Raíces del polinomio característico: [-1.05782739+0.1003839j  -1.05782739-0.1003839j  -1.05456372+0.22323966j
 -1.05456372-0.22323966j -1.01774527+0.34969465j -1.01774527-0.34969465j
 -0.94933175+0.44946613j -0.94933175-0.44946613j -0.85863133+0.62922219j
 -0.85863133-0.62922219j -0.78738236+0.7270708j  -0.78738236-0.7270708j
 -0.69134507+0.81358621j -0.69134507-0.81358621j -0.58329482+0.85241121j
 -0.58329482-0.85241121j -0.41947915+0.97519164j -0.41947915-0.97519164j
 -0.30862303+1.01967798j -0.30862303-1.01967798j -0.18235725+1.04589323j
 -0.18235725-1.04589323j -0.07317524+1.01355735j -0.07317524-1.01355735j
  0.13280061+1.05238283j  0.13280061-1.05238283j  0.25333237

In [15]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p1': -1.555213446084071, 'p2': 0.7751927126085907}

El grado del polinomio es: 2

Raíces del polinomio característico: [1.00311408+0.53269499j 1.00311408-0.53269499j]

Módulo de las raíces: [1.13578246 1.13578246]

¿Las raíces están fuera del círculo unitario?  True

El modelo es invertible

:)


Lo malo es que no es estacionario.

In [45]:
modelo.resid.mean()

-0.011159943451166101

Sus residuos son casi cero, wow!

## **3 MODELO**

Suena loco añadir parámetros para intentar hacer el modelo estacionario, pero puede ser que con los coeficientes adecuados, las raices esten dentro del circulo unitario.

In [46]:
modelo=SARIMAX(zpre,
               order=(11,0,2),
               seasonal_order=(4,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(11, 0, 2)x(4, 1, [], 12)   Log Likelihood                -691.084
Date:                              Sat, 26 Apr 2025   AIC                           1418.169
Time:                                      03:06:59   BIC                           1498.699
Sample:                                           0   HQIC                          1449.409
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.1168      0.239      0.489      0.625      -0.351       0.585
ar.L2         -0.7198      0.216     -3.325      0.001      -1.144      -0.296
ar.L3          0.1394      0.053      2.637      0.008       0.036       0.243
ar.L4         -0.0851      0.056     -1.517      0.129      -0.195       0.025
ar.L5         -0.0911      0.067     -1.362      0.173      -0.222       0.040
ar.L6         -0.1903      0.083     -2.299      0.021      -0.353      -0.028
ar.L7         -0.1130      0.078     -1.457      0.145      -0.265       0.039
ar.L8         -0.1378      0.066     -2.087      0.037      -0.267      -0.008
ar.L9          0.0250      0.055      0.454      0.650      -0.083       0.133
ar.L10        -0.0137      0.051     -0.266      0.790      -0.114       0.087
ar.L11         0.0866      0.044      1.954      0.051      -0.000       0.174
ma.L1         -0.0133      0.240     -0.056      0.956      -0.483       0.456
ma.L2          0.7344      0.213      3.442      0.001       0.316       1.153
ar.S.L12      -0.7054      0.042    -16.926      0.000      -0.787      -0.624
ar.S.L24      -0.5291      0.044    -12.010      0.000      -0.615      -0.443
ar.S.L36      -0.2810      0.043     -6.549      0.000      -0.365      -0.197
ar.S.L48      -0.1452      0.035     -4.204      0.000      -0.213      -0.077
sigma2         0.4877      0.022     22.409      0.000       0.445       0.530
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               194.87
Prob(Q):                              0.99   Prob(JB):                         0.00
Heteroskedasticity (H):               0.85   Skew:                             0.51
Prob(H) (two-sided):                  0.24   Kurtosis:                         5.49
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [48]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.206831,NaN
2,0.206933,NaN
3,0.211279,NaN
4,0.227095,NaN
5,0.391801,NaN
6,0.406829,NaN
7,0.536826,NaN
8,0.870703,NaN
9,1.073577,NaN
10,1.244755,NaN


In [49]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p1': -0.1167552508345661, 'p2': 0.7198064896506363, 'p3': -0.13937576507996408, 'p4': 0.08506198177470811, 'p5': 0.09111832737389458, 'p6': 0.1903172689373877, 'p7': 0.11302640361765769, 'p8': 0.1377956724068651, 'p9': -0.025046054616624666, 'p10': 0.01365996524162506, 'p11': -0.08664543093297898, 'p12': 0.7054095201976057, 'p24': 0.5290690181884172, 'p36': 0.280983915952343, 'p48': 0.14515695218963603}

El grado del polinomio es: 48

Raíces del polinomio característico: [-1.0516124 +0.10868427j -1.0516124 -0.10868427j -1.04132371+0.22614475j
 -1.04132371-0.22614475j -0.99981798+0.34622344j -0.99981798-0.34622344j
 -0.94457079+0.42803742j -0.94457079-0.42803742j -0.84909713+0.62751002j
 -0.84909713-0.62751002j -0.77853657+0.71992772j -0.77853657-0.71992772j
 -0.67781981+0.80484405j -0.67781981-0.80484405j -0.59130052+0.82345508j
 -0.59130052-0.82345508j  1.04865114+0.10983973j  1.04865114-0.10983973j
  1.04020705+0.22644108j  1.04020705-0.22

No funcionó.

In [50]:
modelo.resid.mean()

-0.01263454732792917

Y los residuos crecieron.

## **4 MODELO**

Quitamos la variable más alta.

In [52]:
modelo=SARIMAX(zpre,
               order=(11,0,2),
               seasonal_order=(3,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(11, 0, 2)x(3, 1, [], 12)   Log Likelihood                -695.889
Date:                              Sat, 26 Apr 2025   AIC                           1425.779
Time:                                      03:09:42   BIC                           1501.835
Sample:                                           0   HQIC                          1455.283
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0009      0.059      0.014      0.989      -0.115       0.117
ar.L2         -0.9042      0.058    -15.526      0.000      -1.018      -0.790
ar.L3          0.1546      0.053      2.931      0.003       0.051       0.258
ar.L4         -0.0748      0.057     -1.307      0.191      -0.187       0.037
ar.L5         -0.0916      0.066     -1.388      0.165      -0.221       0.038
ar.L6         -0.2112      0.077     -2.738      0.006      -0.362      -0.060
ar.L7         -0.1455      0.070     -2.067      0.039      -0.283      -0.008
ar.L8         -0.1525      0.071     -2.145      0.032      -0.292      -0.013
ar.L9          0.0305      0.062      0.492      0.623      -0.091       0.152
ar.L10        -0.0349      0.050     -0.700      0.484      -0.133       0.063
ar.L11         0.0847      0.045      1.881      0.060      -0.004       0.173
ma.L1          0.1028      0.050      2.072      0.038       0.006       0.200
ma.L2          0.9331      0.047     19.843      0.000       0.841       1.025
ar.S.L12      -0.6864      0.039    -17.513      0.000      -0.763      -0.610
ar.S.L24      -0.4556      0.041    -11.165      0.000      -0.536      -0.376
ar.S.L36      -0.1718      0.037     -4.695      0.000      -0.244      -0.100
sigma2         0.4957      0.022     22.491      0.000       0.452       0.539
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               165.35
Prob(Q):                              0.95   Prob(JB):                         0.00
Heteroskedasticity (H):               0.85   Skew:                             0.46
Prob(H) (two-sided):                  0.24   Kurtosis:                         5.30
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [53]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p1': -0.0008504306685534905, 'p2': 0.9042074895698359, 'p3': -0.15455358095850727, 'p4': 0.07479102088576456, 'p5': 0.09164357121808639, 'p6': 0.21118087370731212, 'p7': 0.14551480618223642, 'p8': 0.152496788823688, 'p9': -0.030530085193126243, 'p10': 0.03490792806386738, 'p11': -0.08465168927539977, 'p12': 0.6863632312992001, 'p24': 0.455649649418815, 'p36': 0.17184353863182517}

El grado del polinomio es: 36

Raíces del polinomio característico: [ 1.06613035+0.14222197j  1.06613035-0.14222197j  1.0500307 +0.30403229j
  1.0500307 -0.30403229j  0.94621949+0.4134797j   0.94621949-0.4134797j
  0.8205812 +0.65186304j  0.8205812 -0.65186304j  0.75918219+0.76643607j
  0.75918219-0.76643607j  0.62295542+0.83575388j  0.62295542-0.83575388j
  0.3739971 +1.0034725j   0.3739971 -1.0034725j  -1.06633214+0.14352469j
 -1.06633214-0.14352469j -1.04595347+0.29985854j -1.04595347-0.29985854j
 -0.96062745+0.41164465j -0.96062745-0.41164465j -0.84044274+0.667

## **5 MODELO**

Quitamos la variable más alta.

In [54]:
modelo=SARIMAX(zpre,
               order=(11,0,2),
               seasonal_order=(2,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(11, 0, 2)x(2, 1, [], 12)   Log Likelihood                -703.843
Date:                              Sat, 26 Apr 2025   AIC                           1439.686
Time:                                      03:11:45   BIC                           1511.268
Sample:                                           0   HQIC                          1467.455
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0248      0.056      0.440      0.660      -0.086       0.135
ar.L2         -0.8932      0.055    -16.145      0.000      -1.002      -0.785
ar.L3          0.1404      0.054      2.600      0.009       0.035       0.246
ar.L4         -0.0565      0.056     -1.000      0.317      -0.167       0.054
ar.L5         -0.0878      0.065     -1.347      0.178      -0.216       0.040
ar.L6         -0.1741      0.081     -2.152      0.031      -0.333      -0.016
ar.L7         -0.1238      0.071     -1.736      0.083      -0.264       0.016
ar.L8         -0.1230      0.073     -1.681      0.093      -0.266       0.020
ar.L9          0.0377      0.063      0.597      0.550      -0.086       0.161
ar.L10        -0.0063      0.049     -0.130      0.896      -0.102       0.089
ar.L11         0.0924      0.046      2.019      0.043       0.003       0.182
ma.L1          0.0730      0.046      1.589      0.112      -0.017       0.163
ma.L2          0.9278      0.043     21.401      0.000       0.843       1.013
ar.S.L12      -0.6086      0.035    -17.361      0.000      -0.677      -0.540
ar.S.L24      -0.3360      0.032    -10.653      0.000      -0.398      -0.274
sigma2         0.5088      0.023     22.037      0.000       0.464       0.554
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               143.85
Prob(Q):                              1.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.83   Skew:                             0.38
Prob(H) (two-sided):                  0.18   Kurtosis:                         5.18
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [55]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p1': -0.024793315761773772, 'p2': 0.8931526010713633, 'p3': -0.140416802104324, 'p4': 0.05645410687654267, 'p5': 0.08782618157556446, 'p6': 0.1741475076070259, 'p7': 0.1237839148149425, 'p8': 0.12299083817254432, 'p9': -0.0377064539363953, 'p10': 0.006345874788013876, 'p11': -0.09243268115627988, 'p12': 0.6086364269902121, 'p24': 0.3359800269176826}

El grado del polinomio es: 24

Raíces del polinomio característico: [ 1.07578543+0.18826834j  1.07578543-0.18826834j  0.97297415+0.40243319j
  0.97297415-0.40243319j  0.8023425 +0.68189572j  0.8023425 -0.68189572j
  0.65621639+0.82994168j  0.65621639-0.82994168j -1.07540898+0.19271527j
 -1.07540898-0.19271527j -0.98560068+0.39213389j -0.98560068-0.39213389j
 -0.81862896+0.71759603j -0.81862896-0.71759603j -0.61713503+0.82060979j
 -0.61713503-0.82060979j  0.31191254+1.02194317j  0.31191254-1.02194317j
 -0.30624211+1.01944212j -0.30624211-1.01944212j  0.14537911+0.95824338j
  0.14537911-0.95824338

## **6 MODELO**

Quitamos la variable más alta.

In [56]:
modelo=SARIMAX(zpre,
               order=(11,0,2),
               seasonal_order=(1,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(11, 0, 2)x(1, 1, [], 12)   Log Likelihood                -740.734
Date:                              Sat, 26 Apr 2025   AIC                           1511.468
Time:                                      03:12:30   BIC                           1578.576
Sample:                                           0   HQIC                          1537.501
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.7446      0.257     -2.902      0.004      -1.247      -0.242
ar.L2         -0.3207      0.243     -1.320      0.187      -0.797       0.155
ar.L3          0.0986      0.062      1.578      0.115      -0.024       0.221
ar.L4         -0.0180      0.068     -0.266      0.790      -0.151       0.115
ar.L5         -0.1251      0.066     -1.891      0.059      -0.255       0.005
ar.L6         -0.1804      0.092     -1.968      0.049      -0.360      -0.001
ar.L7         -0.0648      0.081     -0.801      0.423      -0.223       0.094
ar.L8         -0.0056      0.059     -0.095      0.924      -0.121       0.110
ar.L9          0.0474      0.061      0.783      0.434      -0.071       0.166
ar.L10         0.0660      0.058      1.141      0.254      -0.047       0.179
ar.L11         0.1529      0.045      3.386      0.001       0.064       0.241
ma.L1          0.8489      0.262      3.239      0.001       0.335       1.363
ma.L2          0.4103      0.264      1.554      0.120      -0.107       0.928
ar.S.L12      -0.3954      0.043     -9.249      0.000      -0.479      -0.312
sigma2         0.5730      0.024     23.917      0.000       0.526       0.620
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):               244.37
Prob(Q):                              0.92   Prob(JB):                         0.00
Heteroskedasticity (H):               0.80   Skew:                             0.38
Prob(H) (two-sided):                  0.10   Kurtosis:                         5.91
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [57]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p1': 0.7445839191513165, 'p2': 0.320662818158104, 'p3': -0.09857131205509115, 'p4': 0.018049616103877023, 'p5': 0.12510575662966333, 'p6': 0.1804231703249629, 'p7': 0.06482580972733076, 'p8': 0.005583895304386813, 'p9': -0.0474331135171728, 'p10': -0.0660031124917558, 'p11': -0.15292138067402267, 'p12': 0.3954393426281018}

El grado del polinomio es: 12

Raíces del polinomio característico: [ 1.16875195+0.3478956j   1.16875195-0.3478956j   0.8219616 +0.77675691j
  0.8219616 -0.77675691j  0.21720539+1.09268664j  0.21720539-1.09268664j
 -0.32047601+0.97068513j -0.32047601-0.97068513j -0.97569533+0.27720844j
 -0.97569533-0.27720844j -0.7183913 +0.69314317j -0.7183913 -0.69314317j]

Módulo de las raíces: [1.21943121 1.21943121 1.13091652 1.13091652 1.11406565 1.11406565
 1.02222037 1.02222037 1.01431055 1.01431055 0.99826525 0.99826525]

¿Las raíces están fuera del círculo unitario?  False

El modelo no es estacionario


## **7 MODELO**

Hay que encontrar un modelo que sea estacionario.

In [66]:
modelo=SARIMAX(zpre,
               order=(0,0,2),
               seasonal_order=(1,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                  660
Model:             SARIMAX(0, 0, 2)x(1, 1, [], 12)   Log Likelihood                -757.615
Date:                             Sat, 26 Apr 2025   AIC                           1523.230
Time:                                     03:13:50   BIC                           1541.126
Sample:                                          0   HQIC                          1530.172
                                             - 660                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1065      0.030      3.573      0.000       0.048       0.165
ma.L2          0.0055      0.036      0.154      0.877      -0.064       0.075
ar.S.L12      -0.4378      0.025    -17.374      0.000      -0.487      -0.388
sigma2         0.6044      0.022     26.912      0.000       0.560       0.648
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               265.82
Prob(Q):                              0.99   Prob(JB):                         0.00
Heteroskedasticity (H):               0.85   Skew:                             0.25
Prob(H) (two-sided):                  0.23   Kurtosis:                         6.10
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [67]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p12': 0.43778901161464395}

El grado del polinomio es: 12

Raíces del polinomio característico: [-1.034757 +0.2772623j -1.034757 -0.2772623j -0.7574947+0.7574947j
 -0.7574947-0.7574947j -0.2772623+1.034757j  -0.2772623-1.034757j
  0.2772623+1.034757j   0.2772623-1.034757j   0.7574947+0.7574947j
  0.7574947-0.7574947j  1.034757 +0.2772623j  1.034757 -0.2772623j]

Módulo de las raíces: [1.07125928 1.07125928 1.07125928 1.07125928 1.07125928 1.07125928
 1.07125928 1.07125928 1.07125928 1.07125928 1.07125928 1.07125928]

¿Las raíces están fuera del círculo unitario?  True

El modelo es estacionario

:)


In [68]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.070283,NaN
2,0.111341,NaN
3,1.692531,NaN
4,4.732164,2.960362e-02
5,16.071586,3.236677e-04
6,21.952927,6.671891e-05
7,22.515661,1.581976e-04
8,22.517688,4.172926e-04
9,23.541992,6.338632e-04
10,26.331309,4.394147e-04


No.

## **8 MODELO**

Hay que encontrar un modelo que sea estacionario.

In [69]:
modelo=SARIMAX(zpre,
               order=(0,0,2),
               seasonal_order=(2,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                  660
Model:             SARIMAX(0, 0, 2)x(2, 1, [], 12)   Log Likelihood                -724.202
Date:                             Sat, 26 Apr 2025   AIC                           1458.404
Time:                                     03:15:10   BIC                           1480.773
Sample:                                          0   HQIC                          1467.081
                                             - 660                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1202      0.030      4.004      0.000       0.061       0.179
ma.L2          0.0184      0.035      0.519      0.603      -0.051       0.088
ar.S.L12      -0.5854      0.026    -22.543      0.000      -0.636      -0.534
ar.S.L24      -0.3267      0.027    -12.162      0.000      -0.379      -0.274
sigma2         0.5429      0.021     25.998      0.000       0.502       0.584
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               196.40
Prob(Q):                              0.98   Prob(JB):                         0.00
Heteroskedasticity (H):               0.86   Skew:                             0.33
Prob(H) (two-sided):                  0.27   Kurtosis:                         5.61
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [70]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p12': 0.585396872333356, 'p24': 0.32670087366234263}

El grado del polinomio es: 24

Raíces del polinomio característico: [-1.03158602+0.18313907j -1.03158602-0.18313907j -0.98494923+0.35718993j
 -0.98494923-0.35718993j -0.80181017+0.67439609j -0.80181017-0.67439609j
 -0.67439609+0.80181017j -0.67439609-0.80181017j -0.35718993+0.98494923j
 -0.35718993-0.98494923j -0.18313907+1.03158602j -0.18313907-1.03158602j
  0.18313907+1.03158602j  0.18313907-1.03158602j  0.35718993+0.98494923j
  0.35718993-0.98494923j  0.67439609+0.80181017j  0.67439609-0.80181017j
  0.80181017+0.67439609j  0.80181017-0.67439609j  1.03158602+0.18313907j
  1.03158602-0.18313907j  0.98494923+0.35718993j  0.98494923-0.35718993j]

Módulo de las raíces: [1.04771639 1.04771639 1.04771639 1.04771639 1.04771639 1.04771639
 1.04771639 1.04771639 1.04771639 1.04771639 1.04771639 1.04771639
 1.04771639 1.04771639 1.04771639 1.04771639 1.04771639 1.04771639
 1.04771639 1.04771639 1

In [71]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.142212,NaN
2,0.167607,NaN
3,1.220539,NaN
4,4.427091,NaN
5,19.615131,9.471611e-06
6,29.220722,4.516488e-07
7,29.239097,1.994755e-06
8,29.490481,6.214021e-06
9,31.144856,8.770112e-06
10,33.621456,7.959647e-06


Empeoró.

## **9 MODELO**

Hay que encontrar un modelo que sea estacionario.

In [72]:
modelo=SARIMAX(zpre,
               order=(0,0,2),
               seasonal_order=(3,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                  660
Model:             SARIMAX(0, 0, 2)x(3, 1, [], 12)   Log Likelihood                -717.536
Date:                             Sat, 26 Apr 2025   AIC                           1447.072
Time:                                     03:16:19   BIC                           1473.915
Sample:                                          0   HQIC                          1457.485
                                             - 660                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1286      0.030      4.259      0.000       0.069       0.188
ma.L2          0.0174      0.037      0.474      0.635      -0.054       0.089
ar.S.L12      -0.6372      0.027    -23.429      0.000      -0.691      -0.584
ar.S.L24      -0.4167      0.032    -12.950      0.000      -0.480      -0.354
ar.S.L36      -0.1513      0.033     -4.520      0.000      -0.217      -0.086
sigma2         0.5311      0.020     26.333      0.000       0.492       0.571
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               213.16
Prob(Q):                              0.98   Prob(JB):                         0.00
Heteroskedasticity (H):               0.89   Skew:                             0.40
Prob(H) (two-sided):                  0.39   Kurtosis:                         5.69
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [73]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p12': 0.6372099437157339, 'p24': 0.41667878504933864, 'p36': 0.15130267228637645}

El grado del polinomio es: 36

Raíces del polinomio característico: [-1.03596194+0.15042229j -1.03596194-0.15042229j -1.03167084+0.27643537j
 -1.03167084-0.27643537j -0.97238051+0.38771144j -0.97238051-0.38771144j
 -0.82195821+0.6482505j  -0.82195821-0.6482505j  -0.75523547+0.75523547j
 -0.75523547-0.75523547j -0.6482505 +0.82195821j -0.6482505 -0.82195821j
 -0.38771144+0.97238051j -0.38771144-0.97238051j -0.27643537+1.03167084j
 -0.27643537-1.03167084j -0.15042229+1.03596194j -0.15042229-1.03596194j
  0.15042229+1.03596194j  0.15042229-1.03596194j  0.27643537+1.03167084j
  0.27643537-1.03167084j  0.38771144+0.97238051j  0.38771144-0.97238051j
  0.6482505 +0.82195821j  0.6482505 -0.82195821j  0.75523547+0.75523547j
  0.75523547-0.75523547j  0.82195821+0.6482505j   0.82195821-0.6482505j
  1.03596194+0.15042229j  1.03596194-0.15042229j  1.03167084+0.27643537j
  

In [74]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.161614,NaN
2,0.199501,NaN
3,1.629878,NaN
4,6.053855,NaN
5,22.675223,NaN
6,34.198111,4.977725e-09
7,34.347012,3.480495e-08
8,34.998434,1.219178e-07
9,36.638606,2.138057e-07
10,39.119495,2.246877e-07


No.

## **10 MODELO**

In [ ]:
modelo=SARIMAX(zpre,
               order=(0,0,3),
               seasonal_order=(4,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                  660
Model:             SARIMAX(0, 0, 3)x(4, 1, [], 12)   Log Likelihood                -710.745
Date:                             Sat, 26 Apr 2025   AIC                           1437.489
Time:                                     03:16:41   BIC                           1473.280
Sample:                                          0   HQIC                          1451.374
                                             - 660                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1452      0.030      4.761      0.000       0.085       0.205
ma.L2          0.0626      0.036      1.724      0.085      -0.009       0.134
ma.L3          0.1042      0.041      2.528      0.011       0.023       0.185
ar.S.L12      -0.6669      0.028    -24.021      0.000      -0.721      -0.612
ar.S.L24      -0.4797      0.034    -13.985      0.000      -0.547      -0.412
ar.S.L36      -0.2454      0.037     -6.605      0.000      -0.318      -0.173
ar.S.L48      -0.1316      0.033     -4.048      0.000      -0.195      -0.068
sigma2         0.5192      0.020     25.923      0.000       0.480       0.558
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):               200.30
Prob(Q):                              0.92   Prob(JB):                         0.00
Heteroskedasticity (H):               0.87   Skew:                             0.39
Prob(H) (two-sided):                  0.30   Kurtosis:                         5.61
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [76]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p12': 0.6668694576201142, 'p24': 0.4796963555595531, 'p36': 0.24541847252267884, 'p48': 0.13162527709134034}

El grado del polinomio es: 48

Raíces del polinomio característico: [-1.03620502+0.11712179j -1.03620502-0.11712179j -1.02166627+0.21233787j
 -1.02166627-0.21233787j -0.99095788+0.32694315j -0.99095788-0.32694315j
 -0.95594076+0.41667206j -0.95594076-0.41667206j -0.83881897+0.61953296j
 -0.83881897-0.61953296j -0.77862001+0.69472312j -0.77862001-0.69472312j
 -0.69472312+0.77862001j -0.69472312-0.77862001j -0.61953296+0.83881897j
 -0.61953296-0.83881897j -0.41667206+0.95594076j -0.41667206-0.95594076j
 -0.32694315+0.99095788j -0.32694315-0.99095788j -0.21233787+1.02166627j
 -0.21233787-1.02166627j -0.11712179+1.03620502j -0.11712179-1.03620502j
  0.11712179+1.03620502j  0.11712179-1.03620502j  0.21233787+1.02166627j
  0.21233787-1.02166627j  0.32694315+0.99095788j  0.32694315-0.99095788j
  0.41667206+0.95594076j  0.41667206-0.95594076

In [77]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.084042,NaN
2,0.212157,NaN
3,0.681979,NaN
4,3.392631,NaN
5,19.000580,NaN
6,31.564746,NaN
7,31.702920,NaN
8,32.270434,1.341390e-08
9,33.846525,4.470134e-08
10,36.586390,5.628392e-08


## **11 MODELO**

Hay que encontrar un modelo que sea estacionario.

In [78]:
modelo=SARIMAX(zpre,
               order=(0,0,2),
               seasonal_order=(6,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                  660
Model:             SARIMAX(0, 0, 2)x(6, 1, [], 12)   Log Likelihood                -707.338
Date:                             Sat, 26 Apr 2025   AIC                           1432.676
Time:                                     03:18:16   BIC                           1472.941
Sample:                                          0   HQIC                          1448.296
                                             - 660                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1688      0.032      5.319      0.000       0.107       0.231
ma.L2          0.0366      0.037      0.989      0.323      -0.036       0.109
ar.S.L12      -0.6836      0.030    -22.739      0.000      -0.742      -0.625
ar.S.L24      -0.5184      0.038    -13.707      0.000      -0.593      -0.444
ar.S.L36      -0.3178      0.045     -6.997      0.000      -0.407      -0.229
ar.S.L48      -0.2392      0.047     -5.059      0.000      -0.332      -0.147
ar.S.L60      -0.1641      0.045     -3.678      0.000      -0.252      -0.077
ar.S.L72      -0.0281      0.041     -0.683      0.494      -0.109       0.053
sigma2         0.5128      0.020     25.979      0.000       0.474       0.552
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               205.67
Prob(Q):                              0.95   Prob(JB):                         0.00
Heteroskedasticity (H):               0.88   Skew:                             0.47
Prob(H) (two-sided):                  0.33   Kurtosis:                         5.59
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [79]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p12': 0.6835605247638168, 'p24': 0.518389786551657, 'p36': 0.3178104587063984, 'p48': 0.23923006774580677, 'p60': 0.16409248920035985, 'p72': 0.028128780024386225}

El grado del polinomio es: 72

Raíces del polinomio característico: [-1.09022043+0.29212368j -1.09022043-0.29212368j -1.02865138+0.08664634j
 -1.02865138-0.08664634j -1.0180959 +0.17499555j -1.0180959 -0.17499555j
 -1.01340076+0.27153992j -1.01340076-0.27153992j -0.96919468+0.35749736j
 -0.96919468-0.35749736j -0.9341614 +0.43928775j -0.9341614 -0.43928775j
 -0.79809675+0.79809675j -0.79809675-0.79809675j -0.84751505+0.58936362j
 -0.84751505-0.58936362j -0.79419913+0.66059854j -0.79419913-0.66059854j
 -0.74186085+0.74186085j -0.74186085-0.74186085j -0.66059854+0.79419913j
 -0.66059854-0.79419913j -0.58936362+0.84751505j -0.58936362-0.84751505j
 -0.29212368+1.09022043j -0.29212368-1.09022043j -0.43928775+0.9341614j
 -0.43928775-0.9341614j  -0.35749736+0.96919468j -0.35749736-0.969

In [80]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.198149,NaN
2,0.238422,NaN
3,1.279700,NaN
4,5.918628,NaN
5,24.598963,NaN
6,39.598849,NaN
7,40.023444,NaN
8,41.080435,NaN
9,42.800647,6.061211e-11
10,46.580707,7.675876e-11


No.

## **12 MODELO**

Hay que encontrar un modelo que sea estacionario.

In [85]:
modelo=SARIMAX(zpre,
               order=(1,0,2),
               seasonal_order=(6,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                  660
Model:             SARIMAX(1, 0, 2)x(6, 1, [], 12)   Log Likelihood                -703.921
Date:                             Sat, 26 Apr 2025   AIC                           1427.842
Time:                                     03:26:12   BIC                           1472.581
Sample:                                          0   HQIC                          1445.197
                                             - 660                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.9196      0.045     20.388      0.000       0.831       1.008
ma.L1         -0.7759      0.054    -14.470      0.000      -0.881      -0.671
ma.L2         -0.1826      0.033     -5.614      0.000      -0.246      -0.119
ar.S.L12      -0.6667      0.031    -21.480      0.000      -0.728      -0.606
ar.S.L24      -0.5022      0.038    -13.242      0.000      -0.576      -0.428
ar.S.L36      -0.3036      0.045     -6.688      0.000      -0.393      -0.215
ar.S.L48      -0.2237      0.047     -4.805      0.000      -0.315      -0.132
ar.S.L60      -0.1532      0.044     -3.498      0.000      -0.239      -0.067
ar.S.L72      -0.0205      0.041     -0.504      0.614      -0.100       0.059
sigma2         0.5072      0.021     24.197      0.000       0.466       0.548
===================================================================================
Ljung-Box (L1) (Q):                   0.07   Jarque-Bera (JB):               174.83
Prob(Q):                              0.79   Prob(JB):                         0.00
Heteroskedasticity (H):               0.89   Skew:                             0.50
Prob(H) (two-sided):                  0.38   Kurtosis:                         5.34
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [87]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p1': -0.9195994212465887, 'p12': 0.6667206444134606, 'p24': 0.5021637027185452, 'p36': 0.30355899688713345, 'p48': 0.22368505051570503, 'p60': 0.1531701725128393, 'p72': 0.020532643013664757}

El grado del polinomio es: 72

Raíces del polinomio característico: [-1.12056325+0.30021477j -1.12056325-0.30021477j -1.03657322+0.08033245j
 -1.03657322-0.08033245j -1.03438789+0.17343022j -1.03438789-0.17343022j
 -1.02493559+0.27761471j -1.02493559-0.27761471j -0.97867734+0.3714574j
 -0.97867734-0.3714574j  -0.93369681+0.44969974j -0.93369681-0.44969974j
 -0.82040208+0.82025627j -0.82040208-0.82025627j -0.85745582+0.59175647j
 -0.85745582-0.59175647j -0.80612476+0.67227075j -0.80612476-0.67227075j
 -0.74377138+0.75604414j -0.74377138-0.75604414j -0.65532979+0.81188457j
 -0.65532979-0.81188457j -0.57962561+0.85335165j -0.57962561-0.85335165j
 -0.30041332+1.12061772j -0.30041332-1.12061772j -0.44385897+0.9437217j
 -0.44385897-0.9437217j  -0.35628568+0.

No.

## **13 MODELO**

La parte AR72 no cumplia el principio de parsimonia, entonces la eliminamos.

In [89]:
modelo=SARIMAX(zpre,
               order=(1,0,2),
               seasonal_order=(5,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                  660
Model:             SARIMAX(1, 0, 2)x(5, 1, [], 12)   Log Likelihood                -704.031
Date:                             Sat, 26 Apr 2025   AIC                           1426.062
Time:                                     03:28:05   BIC                           1466.327
Sample:                                          0   HQIC                          1441.682
                                             - 660                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.9220      0.043     21.444      0.000       0.838       1.006
ma.L1         -0.7796      0.051    -15.217      0.000      -0.880      -0.679
ma.L2         -0.1813      0.032     -5.651      0.000      -0.244      -0.118
ar.S.L12      -0.6634      0.029    -22.771      0.000      -0.720      -0.606
ar.S.L24      -0.4968      0.036    -13.929      0.000      -0.567      -0.427
ar.S.L36      -0.2961      0.042     -7.045      0.000      -0.378      -0.214
ar.S.L48      -0.2124      0.040     -5.350      0.000      -0.290      -0.135
ar.S.L60      -0.1389      0.032     -4.344      0.000      -0.202      -0.076
sigma2         0.5074      0.021     24.212      0.000       0.466       0.548
===================================================================================
Ljung-Box (L1) (Q):                   0.06   Jarque-Bera (JB):               172.88
Prob(Q):                              0.80   Prob(JB):                         0.00
Heteroskedasticity (H):               0.89   Skew:                             0.50
Prob(H) (two-sided):                  0.37   Kurtosis:                         5.33
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [90]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p1': -0.922042080559869, 'p12': 0.6633766318437117, 'p24': 0.49676861984565746, 'p36': 0.2960584170910465, 'p48': 0.2124318749223011, 'p60': 0.13889485062280885}

El grado del polinomio es: 60

Raíces del polinomio característico: [-1.03733552+0.08373407j -1.03733552-0.08373407j -1.03123846+0.1776705j
 -1.03123846-0.1776705j  -1.01640714+0.27499386j -1.01640714-0.27499386j
 -0.9785268 +0.36641426j -0.9785268 -0.36641426j -0.93588411+0.44735176j
 -0.93588411-0.44735176j -0.85626221+0.59528466j -0.85626221-0.59528466j
 -0.80093941+0.67399135j -0.80093941-0.67399135j -0.73828032+0.74920582j
 -0.73828032-0.74920582j -0.65794601+0.8079217j  -0.65794601-0.8079217j
 -0.58236562+0.85244223j -0.58236562-0.85244223j -0.44081829+0.94621033j
 -0.44081829-0.94621033j -0.35094242+0.98533825j -0.35094242-0.98533825j
 -0.25896511+1.01701909j -0.25896511-1.01701909j -0.15895618+1.0248182j
 -0.15895618-1.0248182j  -0.07734023+1.02249618j -0.07734023-1.0224961

No.

## **14 MODELO**

Nos quedamos con el que tenia los p-valores mas altos y era estacionario, intentaremos mejorarlo con la parte MA.

In [91]:
modelo=SARIMAX(zpre,
               order=(0,0,12),
               seasonal_order=(4,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(0, 0, 12)x(4, 1, [], 12)   Log Likelihood                -670.963
Date:                              Sat, 26 Apr 2025   AIC                           1375.925
Time:                                      03:32:32   BIC                           1451.982
Sample:                                           0   HQIC                          1405.430
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.0982      0.150      0.655      0.513      -0.196       0.392
ma.L2          0.0265      0.134      0.197      0.844      -0.237       0.290
ma.L3      -4.313e-05      0.139     -0.000      1.000      -0.273       0.273
ma.L4         -0.0667      0.144     -0.463      0.643      -0.349       0.216
ma.L5         -0.1299      0.135     -0.961      0.337      -0.395       0.135
ma.L6         -0.1508      0.159     -0.948      0.343      -0.462       0.161
ma.L7         -0.1085      0.133     -0.815      0.415      -0.369       0.152
ma.L8         -0.0413      0.143     -0.290      0.772      -0.321       0.238
ma.L9          0.0055      0.136      0.040      0.968      -0.260       0.271
ma.L10         0.0699      0.135      0.518      0.605      -0.195       0.335
ma.L11         0.1394      0.151      0.925      0.355      -0.156       0.435
ma.L12        -0.8294      0.143     -5.802      0.000      -1.110      -0.549
ar.S.L12       0.0730      0.039      1.885      0.059      -0.003       0.149
ar.S.L24       0.0068      0.039      0.174      0.862      -0.069       0.083
ar.S.L36       0.0849      0.038      2.237      0.025       0.010       0.159
ar.S.L48       0.0030      0.043      0.069      0.945      -0.082       0.088
sigma2         0.4437      0.071      6.256      0.000       0.305       0.583
===================================================================================
Ljung-Box (L1) (Q):                   0.83   Jarque-Bera (JB):               310.22
Prob(Q):                              0.36   Prob(JB):                         0.00
Heteroskedasticity (H):               0.83   Skew:                             0.80
Prob(H) (two-sided):                  0.16   Kurtosis:                         5.99
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

Recordamos que es estacionario:

In [92]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p12': -0.07298912451012092, 'p24': -0.006751612596090889, 'p36': -0.08489038377048784, 'p48': -0.0029997437911570514}

El grado del polinomio es: 48

Raíces del polinomio característico: [-1.27608892e+00+0.341927j   -1.27608892e+00-0.341927j
 -9.34161927e-01+0.93416193j -9.34161927e-01-0.93416193j
 -3.41926997e-01+1.27608892j -3.41926997e-01-1.27608892j
 -1.06289584e+00+0.j         -1.05964465e+00+0.18103425j
 -1.05964465e+00-0.18103425j -1.00819631e+00+0.37304206j
 -1.00819631e+00-0.37304206j -9.20494799e-01+0.53144792j
 -9.20494799e-01-0.53144792j -8.27162057e-01+0.68660259j
 -8.27162057e-01-0.68660259j -6.86602588e-01+0.82716206j
 -6.86602588e-01-0.82716206j -5.31447920e-01+0.9204948j
 -5.31447920e-01-0.9204948j  -3.73042060e-01+1.00819631j
 -3.73042060e-01-1.00819631j  3.41926997e-01+1.27608892j
  3.41926997e-01-1.27608892j -1.81034255e-01+1.05964465j
 -1.81034255e-01-1.05964465j  3.89740240e-16+1.06289584j
  3.89740240e-16-1.06289584j  

In [187]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p1': 0.09656765597433159, 'p2': 0.023574371668346694, 'p3': 0.0013314929114937417, 'p4': -0.06463642747357498, 'p5': -0.1283937779168518, 'p6': -0.14757871367268927, 'p7': -0.10701809296856069, 'p8': -0.040396557236526354, 'p9': 0.007491314187193666, 'p10': 0.06659319395249205, 'p11': 0.13756821870172342, 'p12': -0.8268595928818426}

El grado del polinomio es: 12

Raíces del polinomio característico: [ 0.94229718+0.52707905j  0.94229718-0.52707905j  1.00174149+0.j
  0.50366345+0.86578511j  0.50366345-0.86578511j  0.00604352+1.01088315j
  0.00604352-1.01088315j -0.50162478+0.86946439j -0.50162478-0.86946439j
 -1.00028674+0.j         -0.86791957+0.4989509j  -0.86791957-0.4989509j ]

Módulo de las raíces: [1.07969269 1.07969269 1.00174149 1.00162904 1.00162904 1.01090121
 1.01090121 1.00379069 1.00379069 1.00028674 1.00111757 1.00111757]

¿Las raíces están fuera del círculo unitario?  True

El modelo es invertible

:)


In [94]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,2.205997,NaN
2,2.418970,NaN
3,4.282848,NaN
4,4.455433,NaN
5,6.834598,NaN
6,7.379621,NaN
7,9.104567,NaN
8,9.107488,NaN
9,11.172737,NaN
10,11.175311,NaN


In [95]:
modelo.resid.mean()

-0.047325685996212764

In [96]:
ttest_1samp(modelo.resid, 0)

TtestResult(statistic=-1.7684916861519402, pvalue=0.07744131362633193, df=659)

Los residuos son 0.

In [ ]:
het_breuschpagan(modelo.resid, add_constant(np.arange(len(modelo.resid))))

(2.3030923446048623,
 0.12911671422624768,
 2.3041536992355955,
 0.12950883157687618)

Los residuos son constantes.

In [97]:
print(jarque_bera(modelo.resid))
print(lilliefors(modelo.resid))

SignificanceResult(statistic=349.32954931884024, pvalue=1.3933242073904862e-76)
(0.11091051744258162, 0.0009999999999998899)


Los residuos no se distribuyen normal.

In [98]:
normal_relajacion(modelo.resid)

75.30% de los residuos están dentro de ±1σ (esperado ≈ 68%)
94.09% de los residuos están dentro de ±2σ (esperado ≈ 95%)
98.79% de los residuos están dentro de ±3σ (esperado ≈ 99.7%)


Puede ser parecida a la normal.

Es el primer modelo que cumple los supuestos, esperamos mejorarlo.

## **15 MODELO**

Estamos intentando mejorar el modelo para que todos los errores sean independientes.

In [ ]:
modelo=SARIMAX(zpre,
               order=(0,0,11),
               seasonal_order=(4,1,2,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                        SARIMAX Results                                         
================================================================================================
Dep. Variable:                                        y   No. Observations:                  660
Model:             SARIMAX(0, 0, 11)x(4, 1, [1, 2], 12)   Log Likelihood                -684.598
Date:                                  Sat, 26 Apr 2025   AIC                           1405.196
Time:                                          03:43:31   BIC                           1485.726
Sample:                                               0   HQIC                          1436.436
                                                  - 660                                         
Covariance Type:                                    opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1461      0.035      4.127      0.000       0.077       0.215
ma.L2          0.0412      0.037      1.107      0.268      -0.032       0.114
ma.L3          0.0704      0.039      1.817      0.069      -0.006       0.146
ma.L4         -0.0813      0.048     -1.699      0.089      -0.175       0.013
ma.L5         -0.1717      0.055     -3.128      0.002      -0.279      -0.064
ma.L6         -0.1686      0.062     -2.737      0.006      -0.289      -0.048
ma.L7         -0.0664      0.053     -1.252      0.211      -0.170       0.038
ma.L8         -0.0632      0.044     -1.430      0.153      -0.150       0.023
ma.L9          0.0495      0.043      1.142      0.253      -0.035       0.134
ma.L10         0.0840      0.043      1.950      0.051      -0.000       0.168
ma.L11         0.1387      0.042      3.299      0.001       0.056       0.221
ar.S.L12       0.1573      1.157      0.136      0.892      -2.110       2.424
ar.S.L24      -0.0910      0.118     -0.773      0.440      -0.322       0.140
ar.S.L36       0.0269      0.069      0.391      0.696      -0.108       0.162
ar.S.L48      -0.0496      0.071     -0.695      0.487      -0.189       0.090
ma.S.L12      -0.8443      1.159     -0.728      0.466      -3.116       1.428
ma.S.L24       0.0952      0.899      0.106      0.916      -1.666       1.856
sigma2         0.4763      0.020     23.588      0.000       0.437       0.516
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):               240.36
Prob(Q):                              0.93   Prob(JB):                         0.00
Heteroskedasticity (H):               0.82   Skew:                             0.64
Prob(H) (two-sided):                  0.15   Kurtosis:                         5.69
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [106]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p1': 0.14607122809576184, 'p2': 0.041208838849931574, 'p3': 0.07039426453872902, 'p4': -0.08125471026796705, 'p5': -0.17172630206470915, 'p6': -0.16863126142495521, 'p7': -0.06644227919508217, 'p8': -0.06316819860351366, 'p9': 0.04948548090047478, 'p10': 0.08399813994349539, 'p11': 0.13865437308609346, 'p12': -0.844271014465188, 'p24': 0.09522371691912404}

El grado del polinomio es: 24

Raíces del polinomio característico: [ 0.95763169+0.63521222j  0.95763169-0.63521222j  0.99166327+0.49122717j
  0.99166327-0.49122717j  1.17251963+0.j          1.02740425+0.j
  0.5840005 +1.04345307j  0.5840005 -1.04345307j  0.49370764+0.87097519j
  0.49370764-0.87097519j -0.01498088+1.19336153j -0.01498088-1.19336153j
  0.01476179+1.01450566j  0.01476179-1.01450566j -0.59635592+1.03059307j
 -0.59635592-1.03059307j -0.52103435+0.87852703j -0.52103435-0.87852703j
 -1.03626049+0.59183397j -1.03626049-0.59183397j -1.20337742+0.j
 -0.98896595+0.j         -0.876923

## **16 MODELO**

Estamos intentando mejorar el modelo para que todos los errores sean independientes.

In [108]:
modelo=SARIMAX(zpre,
               order=(0,0,11),
               seasonal_order=(4,1,1,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                       SARIMAX Results                                       
=============================================================================================
Dep. Variable:                                     y   No. Observations:                  660
Model:             SARIMAX(0, 0, 11)x(4, 1, [1], 12)   Log Likelihood                -684.620
Date:                               Sat, 26 Apr 2025   AIC                           1403.239
Time:                                       03:47:00   BIC                           1479.295
Sample:                                            0   HQIC                          1432.743
                                               - 660                                         
Covariance Type:                                 opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1464      0.035      4.151      0.000       0.077       0.216
ma.L2          0.0414      0.037      1.113      0.266      -0.032       0.114
ma.L3          0.0707      0.039      1.830      0.067      -0.005       0.146
ma.L4         -0.0818      0.047     -1.722      0.085      -0.175       0.011
ma.L5         -0.1723      0.055     -3.136      0.002      -0.280      -0.065
ma.L6         -0.1691      0.062     -2.748      0.006      -0.290      -0.049
ma.L7         -0.0670      0.053     -1.264      0.206      -0.171       0.037
ma.L8         -0.0633      0.044     -1.432      0.152      -0.150       0.023
ma.L9          0.0494      0.043      1.141      0.254      -0.035       0.134
ma.L10         0.0843      0.043      1.958      0.050   -9.33e-05       0.169
ma.L11         0.1393      0.042      3.344      0.001       0.058       0.221
ar.S.L12       0.0454      0.085      0.536      0.592      -0.121       0.211
ar.S.L24      -0.0731      0.059     -1.230      0.219      -0.190       0.043
ar.S.L36       0.0274      0.054      0.505      0.614      -0.079       0.134
ar.S.L48      -0.0386      0.047     -0.824      0.410      -0.130       0.053
ma.S.L12      -0.7322      0.076     -9.580      0.000      -0.882      -0.582
sigma2         0.4763      0.020     23.624      0.000       0.437       0.516
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):               242.07
Prob(Q):                              0.93   Prob(JB):                         0.00
Heteroskedasticity (H):               0.82   Skew:                             0.65
Prob(H) (two-sided):                  0.15   Kurtosis:                         5.70
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [109]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p1': 0.1464452048308525, 'p2': 0.04144751882807445, 'p3': 0.07068153631979102, 'p4': -0.0817586811097083, 'p5': -0.17228746296287278, 'p6': -0.16910255798268817, 'p7': -0.06698494041052062, 'p8': -0.06330213243291638, 'p9': 0.049433417986961284, 'p10': 0.0842804275996769, 'p11': 0.13927135354990078, 'p12': -0.732160670189706}

El grado del polinomio es: 12

Raíces del polinomio característico: [ 0.9661415 +0.53363093j  0.9661415 -0.53363093j  1.02494772+0.j
  0.49529044+0.8715967j   0.49529044-0.8715967j   0.01278104+1.01485905j
  0.01278104-1.01485905j -0.518823  +0.8786029j  -0.518823  -0.8786029j
 -0.99149849+0.j         -0.87700476+0.50446521j -0.87700476-0.50446521j]

Módulo de las raíces: [1.10371707 1.10371707 1.02494772 1.00249361 1.00249361 1.01493953
 1.01493953 1.02035306 1.02035306 0.99149849 1.0117423  1.0117423 ]

¿Las raíces están fuera del círculo unitario?  False

El modelo no es invertible


Los coeficientes cambian, aunque en ambos haya 12 variables.

## **17 MODELO**

Estamos intentando mejorar el modelo para que todos los errores sean independientes.

In [111]:
modelo=SARIMAX(zpre,
               order=(0,0,4),
               seasonal_order=(4,1,1,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(0, 0, 4)x(4, 1, [1], 12)   Log Likelihood                -706.593
Date:                              Sat, 26 Apr 2025   AIC                           1433.186
Time:                                      03:49:54   BIC                           1477.924
Sample:                                           0   HQIC                          1450.541
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1506      0.033      4.567      0.000       0.086       0.215
ma.L2          0.0498      0.037      1.359      0.174      -0.022       0.122
ma.L3          0.0836      0.042      2.001      0.045       0.002       0.166
ma.L4         -0.0520      0.048     -1.093      0.275      -0.145       0.041
ar.S.L12      -0.1648      0.150     -1.095      0.273      -0.460       0.130
ar.S.L24      -0.1647      0.104     -1.588      0.112      -0.368       0.039
ar.S.L36      -0.0478      0.076     -0.629      0.530      -0.197       0.101
ar.S.L48      -0.0668      0.052     -1.295      0.195      -0.168       0.034
ma.S.L12      -0.5203      0.153     -3.405      0.001      -0.820      -0.221
sigma2         0.5120      0.020     25.530      0.000       0.473       0.551
===================================================================================
Ljung-Box (L1) (Q):                   0.03   Jarque-Bera (JB):               203.87
Prob(Q):                              0.86   Prob(JB):                         0.00
Heteroskedasticity (H):               0.86   Skew:                             0.47
Prob(H) (two-sided):                  0.26   Kurtosis:                         5.58
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [112]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p1': 0.1506201489006478, 'p2': 0.04976889492036576, 'p3': 0.08363494824943037, 'p4': -0.051974629021235105, 'p12': -0.520334376341435}

El grado del polinomio es: 12

Raíces del polinomio característico: [ 1.07607171+0.j          0.92265761+0.54791988j  0.92265761-0.54791988j
  0.50981054+0.92525631j  0.50981054-0.92525631j -0.00589277+1.04536637j
 -0.00589277-1.04536637j -0.5316129 +0.91490438j -0.5316129 -0.91490438j
 -1.03064035+0.j         -0.91767816+0.50980651j -0.91767816-0.50980651j]

Módulo de las raíces: [1.07607171 1.07308586 1.07308586 1.05641187 1.05641187 1.04538298
 1.04538298 1.05814096 1.05814096 1.03064035 1.04977897 1.04977897]

¿Las raíces están fuera del círculo unitario?  True

El modelo es invertible

:)


In [114]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.311289,NaN
2,0.322320,NaN
3,0.552487,NaN
4,1.025351,NaN
5,20.810716,NaN
6,36.435642,NaN
7,36.849912,NaN
8,37.691857,NaN
9,39.103986,NaN
10,42.235415,8.092126e-11


## **18 MODELO**

Estamos intentando mejorar el modelo para que todos los errores sean independientes.

In [115]:
modelo=SARIMAX(zpre,
               order=(0,0,3),
               seasonal_order=(4,1,1,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(0, 0, 3)x(4, 1, [1], 12)   Log Likelihood                -707.094
Date:                              Sat, 26 Apr 2025   AIC                           1432.189
Time:                                      03:52:02   BIC                           1472.454
Sample:                                           0   HQIC                          1447.809
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1682      0.033      5.120      0.000       0.104       0.233
ma.L2          0.0740      0.036      2.050      0.040       0.003       0.145
ma.L3          0.1065      0.041      2.612      0.009       0.027       0.186
ar.S.L12      -0.1582      0.145     -1.095      0.274      -0.442       0.125
ar.S.L24      -0.1580      0.101     -1.571      0.116      -0.355       0.039
ar.S.L36      -0.0428      0.074     -0.580      0.562      -0.187       0.102
ar.S.L48      -0.0672      0.050     -1.342      0.180      -0.165       0.031
ma.S.L12      -0.5309      0.148     -3.599      0.000      -0.820      -0.242
sigma2         0.5127      0.020     25.657      0.000       0.474       0.552
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):               201.92
Prob(Q):                              0.94   Prob(JB):                         0.00
Heteroskedasticity (H):               0.85   Skew:                             0.43
Prob(H) (two-sided):                  0.25   Kurtosis:                         5.60
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [116]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p1': 0.16822375612346252, 'p2': 0.07397696253302198, 'p3': 0.10648608270424571, 'p12': -0.5309122557148496}

El grado del polinomio es: 12

Raíces del polinomio característico: [ 1.08449625+0.j          0.91713012+0.55503727j  0.91713012-0.55503727j
  0.50697801+0.91852647j  0.50697801-0.91852647j -0.00503521+1.04699705j
 -0.00503521-1.04699705j -0.53339851+0.90910924j -0.53339851-0.90910924j
 -1.03341619+0.j         -0.91121444+0.51049618j -0.91121444-0.51049618j]

Módulo de las raíces: [1.08449625 1.07200468 1.07200468 1.04915089 1.04915089 1.04700915
 1.04700915 1.05403681 1.05403681 1.03341619 1.04447025 1.04447025]

¿Las raíces están fuera del círculo unitario?  True

El modelo es invertible

:)


In [117]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.095306,NaN
2,0.232604,NaN
3,0.793379,NaN
4,3.336661,NaN
5,20.289209,NaN
6,34.944286,NaN
7,35.296495,NaN
8,36.137334,NaN
9,37.929844,7.333480e-10
10,41.167700,1.149602e-09


## **19 MODELO**

Estamos intentando mejorar el modelo para que todos los errores sean independientes.

In [118]:
modelo=SARIMAX(zpre,
               order=(0,0,6),
               seasonal_order=(3,1,1,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(0, 0, 6)x(3, 1, [1], 12)   Log Likelihood                -694.561
Date:                              Sat, 26 Apr 2025   AIC                           1411.122
Time:                                      03:53:03   BIC                           1460.335
Sample:                                           0   HQIC                          1430.213
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1326      0.034      3.948      0.000       0.067       0.198
ma.L2          0.0077      0.037      0.210      0.833      -0.064       0.080
ma.L3          0.0426      0.039      1.085      0.278      -0.034       0.119
ma.L4         -0.0787      0.047     -1.688      0.091      -0.170       0.013
ma.L5         -0.1341      0.053     -2.532      0.011      -0.238      -0.030
ma.L6         -0.1477      0.060     -2.466      0.014      -0.265      -0.030
ar.S.L12      -0.0177      0.079     -0.225      0.822      -0.172       0.137
ar.S.L24      -0.0671      0.059     -1.146      0.252      -0.182       0.048
ar.S.L36       0.0242      0.050      0.486      0.627      -0.074       0.122
ma.S.L12      -0.6945      0.074     -9.448      0.000      -0.839      -0.550
sigma2         0.4924      0.021     23.897      0.000       0.452       0.533
===================================================================================
Ljung-Box (L1) (Q):                   0.04   Jarque-Bera (JB):               205.58
Prob(Q):                              0.83   Prob(JB):                         0.00
Heteroskedasticity (H):               0.85   Skew:                             0.59
Prob(H) (two-sided):                  0.24   Kurtosis:                         5.50
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [119]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p1': 0.13264341779565486, 'p2': 0.007748231452451921, 'p3': 0.04257756677165446, 'p4': -0.07872800716570989, 'p5': -0.13411353403361304, 'p6': -0.14769421134550667, 'p12': -0.6945473350217721}

El grado del polinomio es: 12

Raíces del polinomio característico: [ 1.01211438+0.j          0.92828297+0.53195321j  0.92828297-0.53195321j
  0.4829401 +0.89467277j  0.4829401 -0.89467277j  0.00602966+1.0376631j
  0.00602966-1.0376631j  -0.52507318+0.87982584j -0.52507318-0.87982584j
 -0.89579309+0.50592398j -0.89579309-0.50592398j -1.00488729+0.j        ]

Módulo de las raíces: [1.01211438 1.06989882 1.06989882 1.01669588 1.01669588 1.03768061
 1.03768061 1.02459521 1.02459521 1.0287878  1.0287878  1.00488729]

¿Las raíces están fuera del círculo unitario?  True

El modelo es invertible

:)


In [120]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.403548,NaN
2,0.615738,NaN
3,0.629587,NaN
4,0.758596,NaN
5,2.315093,NaN
6,3.120582,NaN
7,5.407323,NaN
8,6.852464,NaN
9,8.841292,NaN
10,13.305347,NaN


## **20 MODELO**

Estamos intentando mejorar el modelo para que todos los errores sean independientes. Se pensó que, como la parte estacional se calcula diferente a la parte estacionaria, y se repite cada 12 periodos, hacer un modelo con SARIMAX(13, 0, 13)x(0, 1, 0, 12)

In [141]:
modelo=SARIMAX(zpre,
               order=(13,0,13),
               seasonal_order=(0,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                       SARIMAX Results                                       
=============================================================================================
Dep. Variable:                                     y   No. Observations:                  660
Model:             SARIMAX(13, 0, 13)x(0, 1, [], 12)   Log Likelihood                -665.561
Date:                               Sat, 26 Apr 2025   AIC                           1385.123
Time:                                       03:56:17   BIC                           1505.918
Sample:                                            0   HQIC                          1431.983
                                               - 660                                         
Covariance Type:                                 opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.3178      0.739      0.430      0.667      -1.131       1.767
ar.L2         -0.0519      0.086     -0.606      0.545      -0.220       0.116
ar.L3          0.0832      0.066      1.269      0.204      -0.045       0.212
ar.L4         -0.0449      0.091     -0.492      0.623      -0.224       0.134
ar.L5         -0.0366      0.085     -0.430      0.667      -0.203       0.130
ar.L6         -0.0084      0.097     -0.087      0.931      -0.198       0.182
ar.L7          0.0794      0.087      0.913      0.361      -0.091       0.250
ar.L8         -0.0085      0.072     -0.117      0.907      -0.150       0.133
ar.L9          0.0695      0.060      1.156      0.248      -0.048       0.187
ar.L10        -0.0425      0.079     -0.539      0.590      -0.197       0.112
ar.L11         0.0186      0.056      0.336      0.737      -0.090       0.127
ar.L12         0.0265      0.053      0.504      0.615      -0.077       0.130
ar.L13        -0.0749      0.053     -1.404      0.160      -0.179       0.030
ma.L1         -0.1717      0.741     -0.232      0.817      -1.624       1.281
ma.L2          0.0084      0.076      0.110      0.912      -0.141       0.158
ma.L3         -0.0049      0.064     -0.076      0.939      -0.130       0.121
ma.L4         -0.0560      0.069     -0.813      0.416      -0.191       0.079
ma.L5         -0.0912      0.076     -1.195      0.232      -0.241       0.058
ma.L6         -0.1081      0.101     -1.072      0.284      -0.306       0.090
ma.L7         -0.0884      0.095     -0.931      0.352      -0.275       0.098
ma.L8         -0.0350      0.090     -0.391      0.696      -0.211       0.141
ma.L9          0.0004      0.065      0.006      0.995      -0.127       0.127
ma.L10         0.0664      0.063      1.047      0.295      -0.058       0.191
ma.L11         0.1307      0.083      1.578      0.114      -0.032       0.293
ma.L12        -0.8361      0.107     -7.797      0.000      -1.046      -0.626
ma.L13         0.2425      0.616      0.394      0.694      -0.964       1.449
sigma2         0.4428      0.025     18.067      0.000       0.395       0.491
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               330.82
Prob(Q):                              0.97   Prob(JB):                         0.00
Heteroskedasticity (H):               0.81   Skew:                             0.83
Prob(H) (two-sided):                  0.13   Kurtosis:                         6.08
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [142]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p1': -0.3177819105647468, 'p2': 0.0518959538518475, 'p3': -0.08318670773339336, 'p4': 0.04488258633563902, 'p5': 0.03658132929567598, 'p6': 0.00839258468850556, 'p7': -0.07943318725604673, 'p8': 0.008503595211218323, 'p9': -0.06947927656192696, 'p10': 0.042475040705376915, 'p11': -0.01863791897837122, 'p12': -0.026519099444974867, 'p13': 0.07487196978492239}

El grado del polinomio es: 13

Raíces del polinomio característico: [-1.35968978+0.j         -1.0733415 +0.53786495j -1.0733415 -0.53786495j
 -0.61437445+1.0118183j  -0.61437445-1.0118183j  -0.0903893 +1.19274931j
 -0.0903893 -1.19274931j  0.52311146+1.16274935j  0.52311146-1.16274935j
  1.21357637+0.22497121j  1.21357637-0.22497121j  0.89835862+0.75202859j
  0.89835862-0.75202859j]

Módulo de las raíces: [1.35968978 1.20056682 1.20056682 1.18373656 1.18373656 1.19616936
 1.19616936 1.27500261 1.27500261 1.23425267 1.23425267 1.17157808
 1.17157808]

¿Las raíces están fuera del círculo 

In [143]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p1': -0.17170742771632796, 'p2': 0.00837987539608711, 'p3': -0.004866529009096059, 'p4': -0.056048034589391976, 'p5': -0.09116076890591945, 'p6': -0.10814530066710958, 'p7': -0.08842956818636406, 'p8': -0.03502466658535272, 'p9': 0.0003800666587888031, 'p10': 0.06641442368990329, 'p11': 0.13074900847065107, 'p12': -0.8361032135424543, 'p13': 0.24253370814714897}

El grado del polinomio es: 13

Raíces del polinomio característico: [ 3.25887339+0.j          0.94663413+0.54476454j  0.94663413-0.54476454j
  1.00767635+0.j          0.51121499+0.87018246j  0.51121499-0.87018246j
  0.00327921+1.01014688j  0.00327921-1.01014688j -0.50128227+0.86981616j
 -0.50128227-0.86981616j -1.00157375+0.j         -0.8686495 +0.49866703j
 -0.8686495 -0.49866703j]

Módulo de las raíces: [3.25887339 1.09219256 1.09219256 1.00767635 1.00923648 1.00923648
 1.0101522  1.0101522  1.00392433 1.00392433 1.00157375 1.00160908
 1.00160908]

¿Las raíces están fuera del círcul

In [144]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.309415,NaN
2,0.340939,NaN
3,0.389159,NaN
4,0.486721,NaN
5,0.909617,NaN
6,1.060588,NaN
7,1.075078,NaN
8,1.119236,NaN
9,1.131526,NaN
10,1.250239,NaN


Todos los residuos son independientes.

In [145]:
modelo.resid.mean()

-0.03081197425810398

Los residuos son un poco mejores que el modelo 14.

In [146]:
ttest_1samp(modelo.resid, 0)

TtestResult(statistic=-1.1598588418779263, pvalue=0.24652632569558622, df=659)

La media de los residuos es cero.

In [147]:
het_breuschpagan(modelo.resid, add_constant(np.arange(len(modelo.resid))))

(2.2689109396271667,
 0.1319926269452356,
 2.2698385755301165,
 0.13239355151760301)

Los residuos son constantes.

In [148]:
print(jarque_bera(modelo.resid))
print(lilliefors(modelo.resid))

SignificanceResult(statistic=361.99581793289536, pvalue=2.4751806421435983e-79)
(0.11107276226288487, 0.0009999999999998899)


No se distribuye normal.

In [149]:
normal_relajacion(modelo.resid)

76.21% de los residuos están dentro de ±1σ (esperado ≈ 68%)
94.09% de los residuos están dentro de ±2σ (esperado ≈ 95%)
98.79% de los residuos están dentro de ±3σ (esperado ≈ 99.7%)


Parece que tienes que escoger entre tener parte estacional autorregresiva y no tener parte autorregresiva, y visceversa. 

## **21 MODELO**

Estamos intentando mejorar el modelo para que todos los errores sean independientes. Se pensó que, como la parte estacional se calcula diferente a la parte estacionaria, y se repite cada 12 periodos, hacer un modelo con SARIMAX(13, 0, 13)x(0, 1, 0, 12)

In [150]:
modelo=SARIMAX(zpre,
               order=(12,0,12),
               seasonal_order=(0,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                       SARIMAX Results                                       
=============================================================================================
Dep. Variable:                                     y   No. Observations:                  660
Model:             SARIMAX(12, 0, 12)x(0, 1, [], 12)   Log Likelihood                -668.075
Date:                               Sat, 26 Apr 2025   AIC                           1386.150
Time:                                       04:03:10   BIC                           1497.997
Sample:                                            0   HQIC                          1429.539
                                               - 660                                         
Covariance Type:                                 opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0880      0.063      1.396      0.163      -0.036       0.212
ar.L2         -0.0039      0.059     -0.067      0.947      -0.120       0.112
ar.L3          0.0618      0.066      0.937      0.349      -0.067       0.191
ar.L4         -0.0598      0.070     -0.856      0.392      -0.197       0.077
ar.L5         -0.0865      0.077     -1.123      0.261      -0.238       0.064
ar.L6         -0.0619      0.079     -0.779      0.436      -0.218       0.094
ar.L7          0.0485      0.071      0.685      0.493      -0.090       0.187
ar.L8         -0.0191      0.060     -0.316      0.752      -0.137       0.099
ar.L9          0.0611      0.060      1.020      0.308      -0.056       0.179
ar.L10        -0.0104      0.057     -0.184      0.854      -0.121       0.100
ar.L11         0.0517      0.052      0.987      0.324      -0.051       0.154
ar.L12         0.0779      0.045      1.725      0.085      -0.011       0.166
ma.L1          0.0515      0.056      0.922      0.357      -0.058       0.161
ma.L2         -0.0015      0.053     -0.029      0.977      -0.106       0.103
ma.L3          0.0084      0.059      0.142      0.887      -0.107       0.124
ma.L4         -0.0239      0.059     -0.409      0.683      -0.139       0.091
ma.L5         -0.0671      0.060     -1.109      0.267      -0.186       0.051
ma.L6         -0.0919      0.059     -1.549      0.121      -0.208       0.024
ma.L7         -0.0770      0.056     -1.367      0.172      -0.187       0.033
ma.L8         -0.0295      0.055     -0.538      0.590      -0.137       0.078
ma.L9         -0.0103      0.054     -0.188      0.851      -0.117       0.096
ma.L10         0.0354      0.055      0.649      0.516      -0.071       0.142
ma.L11         0.0868      0.057      1.515      0.130      -0.025       0.199
ma.L12        -0.8495      0.055    -15.307      0.000      -0.958      -0.741
sigma2         0.4463      0.022     20.511      0.000       0.404       0.489
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               362.04
Prob(Q):                              0.99   Prob(JB):                         0.00
Heteroskedasticity (H):               0.81   Skew:                             0.84
Prob(H) (two-sided):                  0.12   Kurtosis:                         6.26
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [151]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p1': -0.08801500612137014, 'p2': 0.003948289188681289, 'p3': -0.06180805803462757, 'p4': 0.05982583735511234, 'p5': 0.08652481900991361, 'p6': 0.06194122312482241, 'p7': -0.04849220004230152, 'p8': 0.019054065153567865, 'p9': -0.06111206417121281, 'p10': 0.010369355023939692, 'p11': -0.05168635684498128, 'p12': -0.07786690143903036}

El grado del polinomio es: 12

Raíces del polinomio característico: [-1.51427088+0.j         -1.11037727+0.53564333j -1.11037727-0.53564333j
 -0.61160682+1.05363653j -0.61160682-1.05363653j -0.05676767+1.22144332j
 -0.05676767-1.22144332j  0.6385907 +1.09352899j  0.6385907 -1.09352899j
  1.19002025+0.j          0.97039724+0.61319871j  0.97039724-0.61319871j]

Módulo de las raíces: [1.51427088 1.23282256 1.23282256 1.21828274 1.21828274 1.22276177
 1.22276177 1.26633477 1.26633477 1.19002025 1.14790394 1.14790394]

¿Las raíces están fuera del círculo unitario?  True

El modelo es estacionario

:)


In [152]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p1': 0.05150693148303209, 'p2': -0.0015447352152891392, 'p3': 0.008350753074788355, 'p4': -0.023945328979989022, 'p5': -0.06705549881575146, 'p6': -0.09188617047514569, 'p7': -0.07696705923700708, 'p8': -0.02948825316068973, 'p9': -0.010254017696486222, 'p10': 0.035374265834134735, 'p11': 0.0868239846604158, 'p12': -0.8494912391646735}

El grado del polinomio es: 12

Raíces del polinomio característico: [ 1.00290389+0.j          0.91132643+0.52206335j  0.91132643-0.52206335j
  0.5091592 +0.86705998j  0.5091592 -0.86705998j  0.00465945+1.01365601j
  0.00465945-1.01365601j -0.50264053+0.87158739j -0.50264053-0.87158739j
 -1.00422072+0.j         -0.87074262+0.49938717j -0.87074262-0.49938717j]

Módulo de las raíces: [1.00290389 1.05026949 1.05026949 1.00550291 1.00550291 1.01366671
 1.01366671 1.0061372  1.0061372  1.00422072 1.00378297 1.00378297]

¿Las raíces están fuera del círculo unitario?  True

El modelo es invertible

:)


In [153]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.322680,NaN
2,0.326760,NaN
3,0.382182,NaN
4,0.454116,NaN
5,0.657644,NaN
6,0.676272,NaN
7,0.678431,NaN
8,0.726238,NaN
9,0.776381,NaN
10,0.982696,NaN


Todos los residuos son independientes.

In [154]:
modelo.resid.mean()

-0.03884897661905576

Los residuos son un poco mejores que el modelo 14.

In [155]:
ttest_1samp(modelo.resid, 0)

TtestResult(statistic=-1.4581608315441028, pvalue=0.1452726166485534, df=659)

La media de los residuos es cero.

In [156]:
het_breuschpagan(modelo.resid, add_constant(np.arange(len(modelo.resid))))

(2.115803844333697, 0.14578548854537715, 2.116176277993706, 0.1462268830069441)

Los residuos son constantes.

In [157]:
print(jarque_bera(modelo.resid))
print(lilliefors(modelo.resid))

SignificanceResult(statistic=392.138469694565, pvalue=7.05039152120891e-86)
(0.10034777489083546, 0.0009999999999998899)


No se distribuye normal.

In [158]:
normal_relajacion(modelo.resid)

75.15% de los residuos están dentro de ±1σ (esperado ≈ 68%)
93.94% de los residuos están dentro de ±2σ (esperado ≈ 95%)
98.64% de los residuos están dentro de ±3σ (esperado ≈ 99.7%)


## **22 MODELO**

Estamos intentando mejorar el modelo para que todos los errores sean independientes. Se pensó que, como la parte estacional se calcula diferente a la parte estacionaria, y se repite cada 12 periodos, hacer un modelo con SARIMAX(13, 0, 13)x(0, 1, 0, 12)

In [159]:
modelo=SARIMAX(zpre,
               order=(11,0,11),
               seasonal_order=(0,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                       SARIMAX Results                                       
=============================================================================================
Dep. Variable:                                     y   No. Observations:                  660
Model:             SARIMAX(11, 0, 11)x(0, 1, [], 12)   Log Likelihood                -680.182
Date:                               Sat, 26 Apr 2025   AIC                           1406.365
Time:                                       04:04:01   BIC                           1509.264
Sample:                                            0   HQIC                          1446.283
                                               - 660                                         
Covariance Type:                                 opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.6875      1.329     -0.518      0.605      -3.291       1.916
ar.L2         -0.3365      2.040     -0.165      0.869      -4.336       3.663
ar.L3          0.1506      2.175      0.069      0.945      -4.113       4.414
ar.L4          0.4369      1.639      0.266      0.790      -2.776       3.650
ar.L5          0.5077      0.808      0.628      0.530      -1.076       2.091
ar.L6          0.4240      0.063      6.681      0.000       0.300       0.548
ar.L7          0.3230      0.586      0.551      0.582      -0.826       1.472
ar.L8          0.1030      0.936      0.110      0.912      -1.732       1.938
ar.L9         -0.0434      0.936     -0.046      0.963      -1.878       1.791
ar.L10        -0.2083      0.719     -0.290      0.772      -1.617       1.201
ar.L11        -0.1796      0.311     -0.577      0.564      -0.790       0.430
ma.L1          0.8750      1.333      0.656      0.512      -1.738       3.488
ma.L2          0.5128      2.292      0.224      0.823      -3.980       5.006
ma.L3          0.0351      2.639      0.013      0.989      -5.136       5.207
ma.L4         -0.4597      2.296     -0.200      0.841      -4.960       4.041
ma.L5         -0.8267      1.322     -0.625      0.532      -3.417       1.764
ma.L6         -0.9758      0.196     -4.977      0.000      -1.360      -0.592
ma.L7         -0.8747      1.361     -0.643      0.521      -3.543       1.793
ma.L8         -0.5272      2.286     -0.231      0.818      -5.008       3.953
ma.L9         -0.0450      2.614     -0.017      0.986      -5.168       5.078
ma.L10         0.4576      2.275      0.201      0.841      -4.001       4.917
ma.L11         0.8329      1.317      0.633      0.527      -1.748       3.414
sigma2         0.4545      0.095      4.784      0.000       0.268       0.641
===================================================================================
Ljung-Box (L1) (Q):                   0.17   Jarque-Bera (JB):               283.90
Prob(Q):                              0.68   Prob(JB):                         0.00
Heteroskedasticity (H):               0.80   Skew:                             0.72
Prob(H) (two-sided):                  0.09   Kurtosis:                         5.91
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [160]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p1': 0.6875377547028465, 'p2': 0.3364695072379476, 'p3': -0.15058502520237754, 'p4': -0.4368801472382916, 'p5': -0.5076841675645567, 'p6': -0.4240244566377963, 'p7': -0.32301167471114633, 'p8': -0.10304623345522845, 'p9': 0.04342358659627563, 'p10': 0.20832431135590934, 'p11': 0.1795784496947943}

El grado del polinomio es: 11

Raíces del polinomio característico: [-1.2967674 +0.j         -1.04459283+0.54309488j -1.04459283-0.54309488j
 -0.5835851 +0.98843929j -0.5835851 -0.98843929j -0.0476914 +1.12794997j
 -0.0476914 -1.12794997j  0.56585462+1.00607155j  0.56585462-1.00607155j
  1.24115425+0.j          1.11556845+0.j        ]

Módulo de las raíces: [1.2967674  1.17733862 1.17733862 1.14786053 1.14786053 1.12895775
 1.12895775 1.15428394 1.15428394 1.24115425 1.11556845]

¿Las raíces están fuera del círculo unitario?  True

El modelo es estacionario

:)


In [161]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p1': 0.8749616020302327, 'p2': 0.5127907697801983, 'p3': 0.035108131719792324, 'p4': -0.45970349146997735, 'p5': -0.8267104343584486, 'p6': -0.9757924024961929, 'p7': -0.8747182028221141, 'p8': -0.5272249664371678, 'p9': -0.04502316595243694, 'p10': 0.45762681377357006, 'p11': 0.8329211688632158}

El grado del polinomio es: 11

Raíces del polinomio característico: [-1.00190882e+00+0.j         -8.68464082e-01+0.49809596j
 -8.68464082e-01-0.49809596j -5.00655955e-01+0.86880963j
 -5.00655955e-01-0.86880963j  8.34801553e-04+1.00215915j
  8.34801553e-04-1.00215915j  5.02839195e-01+0.86457006j
  5.02839195e-01-0.86457006j  1.18275633e+00+0.j
  1.00062066e+00+0.j        ]

Módulo de las raíces: [1.00190882 1.00116405 1.00116405 1.00273952 1.00273952 1.0021595
 1.0021595  1.00016431 1.00016431 1.18275633 1.00062066]

¿Las raíces están fuera del círculo unitario?  True

El modelo es invertible

:)


In [162]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.016969,NaN
2,0.224448,NaN
3,0.671097,NaN
4,0.914024,NaN
5,0.914973,NaN
6,1.527478,NaN
7,3.694281,NaN
8,5.974856,NaN
9,7.246020,NaN
10,7.564344,NaN


Empeoró mucho el modelo.

## **23 MODELO**

In [163]:
modelo=SARIMAX(zpre,
               order=(13,0,12),
               seasonal_order=(0,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                       SARIMAX Results                                       
=============================================================================================
Dep. Variable:                                     y   No. Observations:                  660
Model:             SARIMAX(13, 0, 12)x(0, 1, [], 12)   Log Likelihood                -666.280
Date:                               Sat, 26 Apr 2025   AIC                           1384.559
Time:                                       04:05:58   BIC                           1500.880
Sample:                                            0   HQIC                          1429.683
                                               - 660                                         
Covariance Type:                                 opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0750      0.065      1.152      0.249      -0.053       0.202
ar.L2          0.0105      0.057      0.183      0.855      -0.102       0.123
ar.L3          0.0835      0.062      1.356      0.175      -0.037       0.204
ar.L4         -0.0336      0.069     -0.488      0.625      -0.168       0.101
ar.L5         -0.0622      0.076     -0.822      0.411      -0.211       0.086
ar.L6         -0.0412      0.078     -0.528      0.597      -0.194       0.112
ar.L7          0.0486      0.069      0.704      0.481      -0.087       0.184
ar.L8         -0.0180      0.059     -0.303      0.762      -0.134       0.098
ar.L9          0.0578      0.058      0.990      0.322      -0.057       0.172
ar.L10        -0.0202      0.055     -0.366      0.715      -0.128       0.088
ar.L11         0.0225      0.049      0.458      0.647      -0.074       0.119
ar.L12         0.0449      0.044      1.010      0.313      -0.042       0.132
ar.L13        -0.0436      0.046     -0.957      0.339      -0.133       0.046
ma.L1          0.0659      0.060      1.095      0.274      -0.052       0.184
ma.L2         -0.0110      0.053     -0.210      0.834      -0.114       0.092
ma.L3         -0.0161      0.055     -0.294      0.769      -0.124       0.091
ma.L4         -0.0536      0.058     -0.927      0.354      -0.167       0.060
ma.L5         -0.0886      0.059     -1.501      0.133      -0.204       0.027
ma.L6         -0.1035      0.062     -1.676      0.094      -0.225       0.018
ma.L7         -0.0781      0.058     -1.356      0.175      -0.191       0.035
ma.L8         -0.0212      0.055     -0.386      0.700      -0.129       0.086
ma.L9          0.0057      0.054      0.106      0.915      -0.100       0.111
ma.L10         0.0573      0.054      1.064      0.287      -0.048       0.163
ma.L11         0.1180      0.057      2.076      0.038       0.007       0.229
ma.L12        -0.8301      0.055    -15.158      0.000      -0.937      -0.723
sigma2         0.4411      0.022     19.778      0.000       0.397       0.485
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               366.20
Prob(Q):                              0.98   Prob(JB):                         0.00
Heteroskedasticity (H):               0.81   Skew:                             0.86
Prob(H) (two-sided):                  0.12   Kurtosis:                         6.26
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [165]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p1': -0.07496457102962477, 'p2': -0.010526072157830805, 'p3': -0.08348927995092362, 'p4': 0.03357941938803479, 'p5': 0.062225081889355036, 'p6': 0.04120625752548046, 'p7': -0.04864610238393563, 'p8': 0.017969902116988134, 'p9': -0.05776657238271834, 'p10': 0.02015338648577602, 'p11': -0.02254548641813857, 'p12': -0.04487753123736985, 'p13': 0.04364095586994825}

El grado del polinomio es: 13

Raíces del polinomio característico: [ 1.43938312+0.15564306j  1.43938312-0.15564306j  0.99303601+0.72918234j
  0.99303601-0.72918234j  0.55778706+1.19511669j  0.55778706-1.19511669j
 -0.10255982+1.21914394j -0.10255982-1.21914394j -0.62417817+1.01823927j
 -0.62417817-1.01823927j -1.3522147 +0.j         -1.07319325+0.53147464j
 -1.07319325-0.53147464j]

Módulo de las raíces: [1.44777365 1.44777365 1.23200139 1.23200139 1.31887464 1.31887464
 1.22345023 1.22345023 1.19432391 1.19432391 1.3522147  1.19758467
 1.19758467]

¿Las raíces están fuera del círcu

In [166]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p1': 0.06590272143887188, 'p2': -0.011024889317803155, 'p3': -0.01612257185966618, 'p4': -0.05357885398942669, 'p5': -0.08857918142931759, 'p6': -0.10353028409484422, 'p7': -0.07808123229920844, 'p8': -0.02118724924935536, 'p9': 0.005717925820057392, 'p10': 0.05727493910835614, 'p11': 0.11798908887469745, 'p12': -0.8301257226492381}

El grado del polinomio es: 12

Raíces del polinomio característico: [ 1.00435165+0.j          0.92745768+0.51639329j  0.92745768-0.51639329j
  0.51022949+0.86961395j  0.51022949-0.86961395j  0.00465819+1.01339505j
  0.00465819-1.01339505j -0.50207337+0.87113656j -0.50207337-0.87113656j
 -1.00282829+0.j         -0.86996667+0.49880981j -0.86996667-0.49880981j]

Módulo de las raíces: [1.00435165 1.06152709 1.06152709 1.00824727 1.00824727 1.01340576
 1.01340576 1.00546336 1.00546336 1.00282829 1.00282263 1.00282263]

¿Las raíces están fuera del círculo unitario?  True

El modelo es invertible

:)


In [164]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.302711,NaN
2,0.305724,NaN
3,0.336077,NaN
4,0.405474,NaN
5,0.855309,NaN
6,1.093915,NaN
7,1.094445,NaN
8,1.118689,NaN
9,1.133005,NaN
10,1.284532,NaN


Todos los residuos son independientes.

In [167]:
modelo.resid.mean()

-0.038004130559285504

Los residuos son un poco mejores que el modelo 14.

In [168]:
ttest_1samp(modelo.resid, 0)

TtestResult(statistic=-1.4300623750501664, pvalue=0.15317304632058404, df=659)

La media de los residuos es cero.

In [169]:
het_breuschpagan(modelo.resid, add_constant(np.arange(len(modelo.resid))))

(2.1105437492349055,
 0.1462873299862606,
 2.1108983793581793,
 0.14673014231148307)

Los residuos son constantes.

In [170]:
print(jarque_bera(modelo.resid))
print(lilliefors(modelo.resid))

SignificanceResult(statistic=396.5588832824966, pvalue=7.732725975226483e-87)
(0.10592598093369521, 0.0009999999999998899)


No se distribuye normal.

In [171]:
normal_relajacion(modelo.resid)

75.45% de los residuos están dentro de ±1σ (esperado ≈ 68%)
94.24% de los residuos están dentro de ±2σ (esperado ≈ 95%)
98.64% de los residuos están dentro de ±3σ (esperado ≈ 99.7%)


Puede ser una normal.

## **24 MODELO**

In [176]:
modelo=SARIMAX(zpre,
               order=(18,0,12),
               seasonal_order=(0,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                       SARIMAX Results                                       
=============================================================================================
Dep. Variable:                                     y   No. Observations:                  660
Model:             SARIMAX(18, 0, 12)x(0, 1, [], 12)   Log Likelihood                -664.916
Date:                               Sat, 26 Apr 2025   AIC                           1391.831
Time:                                       04:08:29   BIC                           1530.522
Sample:                                            0   HQIC                          1445.634
                                               - 660                                         
Covariance Type:                                 opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.1143      0.066      1.735      0.083      -0.015       0.244
ar.L2          0.0414      0.067      0.615      0.539      -0.091       0.174
ar.L3          0.1172      0.069      1.691      0.091      -0.019       0.253
ar.L4         -0.0313      0.080     -0.390      0.696      -0.189       0.126
ar.L5         -0.0904      0.083     -1.096      0.273      -0.252       0.071
ar.L6         -0.0780      0.082     -0.956      0.339      -0.238       0.082
ar.L7          0.0294      0.074      0.400      0.689      -0.115       0.173
ar.L8         -0.0271      0.062     -0.438      0.661      -0.148       0.094
ar.L9          0.0499      0.062      0.804      0.422      -0.072       0.172
ar.L10        -0.0294      0.056     -0.526      0.599      -0.139       0.080
ar.L11         0.0194      0.050      0.390      0.697      -0.078       0.117
ar.L12         0.0567      0.045      1.270      0.204      -0.031       0.144
ar.L13        -0.0306      0.047     -0.654      0.513      -0.122       0.061
ar.L14         0.0257      0.053      0.481      0.631      -0.079       0.130
ar.L15         0.0303      0.054      0.560      0.575      -0.076       0.136
ar.L16         0.0319      0.067      0.479      0.632      -0.099       0.163
ar.L17        -0.0646      0.078     -0.825      0.410      -0.218       0.089
ar.L18        -0.0224      0.078     -0.289      0.773      -0.175       0.130
ma.L1          0.0322      0.061      0.532      0.595      -0.086       0.151
ma.L2         -0.0475      0.064     -0.738      0.460      -0.174       0.079
ma.L3         -0.0522      0.065     -0.806      0.420      -0.179       0.075
ma.L4         -0.0658      0.072     -0.915      0.360      -0.207       0.075
ma.L5         -0.0596      0.076     -0.789      0.430      -0.208       0.088
ma.L6         -0.0597      0.072     -0.825      0.409      -0.202       0.082
ma.L7         -0.0413      0.065     -0.634      0.526      -0.169       0.086
ma.L8          0.0031      0.064      0.048      0.962      -0.123       0.129
ma.L9          0.0236      0.067      0.352      0.725      -0.108       0.155
ma.L10         0.0678      0.065      1.037      0.300      -0.060       0.196
ma.L11         0.1125      0.063      1.774      0.076      -0.012       0.237
ma.L12        -0.8599      0.059    -14.453      0.000      -0.976      -0.743
sigma2         0.4396      0.023     19.305      0.000       0.395       0.484
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):               370.10
Prob(Q):                              0.93   Prob(JB):                         0.00
Heteroskedasticity (H):               0.82   Skew:                             0.86
Prob(H) (two-sided):                  0.14   Kurtosis:                         6.27


In [179]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p1': -0.11432539901926293, 'p2': -0.04144058544514361, 'p3': -0.11721772371478828, 'p4': 0.03133134265323654, 'p5': 0.09041095397218918, 'p6': 0.07796921164311178, 'p7': -0.02941570271165147, 'p8': 0.02713015585621766, 'p9': -0.04990913915978519, 'p10': 0.02943029116653718, 'p11': -0.019367108602501168, 'p12': -0.05667106865253166, 'p13': 0.030556942872824493, 'p14': -0.025674654806970978, 'p15': -0.0303109614753607, 'p16': -0.03192525962334265, 'p17': 0.06456586116665586, 'p18': 0.02244096831728789}

El grado del polinomio es: 18

Raíces del polinomio característico: [-3.23936984+0.j         -1.20203096+0.j         -1.04424535+0.46727377j
 -1.04424535-0.46727377j -0.87985861+0.89690005j -0.87985861-0.89690005j
 -0.51483362+1.02568698j -0.51483362-1.02568698j -0.08139583+1.11673802j
 -0.08139583-1.11673802j  0.36757588+1.11139395j  0.36757588-1.11139395j
  0.7627411 +0.87721935j  0.7627411 -0.87721935j  1.19625145+0.17202524j
  1.19625145-0.

In [178]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p1': 0.03219029649942279, 'p2': -0.047504637291879453, 'p3': -0.052195715536166865, 'p4': -0.06578548511160859, 'p5': -0.05960185330849227, 'p6': -0.059748197268223446, 'p7': -0.04125905015501252, 'p8': 0.0030916918824600848, 'p9': 0.023612749823060955, 'p10': 0.06780315654482219, 'p11': 0.11247574763089202, 'p12': -0.8598691272386679}

El grado del polinomio es: 12

Raíces del polinomio característico: [ 1.00539336+0.j          0.91749514+0.49628245j  0.91749514-0.49628245j
  0.51436193+0.87224478j  0.51436193-0.87224478j  0.00264355+1.01007516j
  0.00264355-1.01007516j -0.50100426+0.87053986j -0.50100426-0.87053986j
 -1.00203362+0.j         -0.8697734 +0.49871886j -0.8697734 -0.49871886j]

Módulo de las raíces: [1.00539336 1.04311725 1.04311725 1.01261007 1.01261007 1.01007862
 1.01007862 1.00441272 1.00441272 1.00203362 1.00260973 1.00260973]

¿Las raíces están fuera del círculo unitario?  True

El modelo es invertible

:)


In [177]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.243281,NaN
2,0.243443,NaN
3,0.295042,NaN
4,0.353055,NaN
5,0.930364,NaN
6,1.120422,NaN
7,1.121360,NaN
8,1.170988,NaN
9,1.200009,NaN
10,1.324741,NaN


Todos los residuos son independientes.

In [180]:
modelo.resid.mean()

-0.03524056874540263

Los residuos son un poco mejores que el modelo 14.

In [181]:
ttest_1samp(modelo.resid, 0)

TtestResult(statistic=-1.3288028558257785, pvalue=0.18437292900259222, df=659)

La media de los residuos es cero.

In [182]:
het_breuschpagan(modelo.resid, add_constant(np.arange(len(modelo.resid))))

(1.9660608958480341,
 0.16086659713281803,
 1.9659594932583788,
 0.1613489620593532)

Los residuos son constantes.

In [183]:
print(jarque_bera(modelo.resid))
print(lilliefors(modelo.resid))

SignificanceResult(statistic=397.0448229769898, pvalue=6.064739572011363e-87)
(0.10215596764675039, 0.0009999999999998899)


No se distribuye normal.

In [184]:
normal_relajacion(modelo.resid)

75.76% de los residuos están dentro de ±1σ (esperado ≈ 68%)
94.09% de los residuos están dentro de ±2σ (esperado ≈ 95%)
98.94% de los residuos están dentro de ±3σ (esperado ≈ 99.7%)


## **25 MODELO**

Se busca mejorar el modelo 14 cambiando en una unidad las variables que se utilizarán.

In [186]:
modelo=SARIMAX(zpre,
               order=(0,0,12),
               seasonal_order=(5,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(0, 0, 12)x(5, 1, [], 12)   Log Likelihood                -670.812
Date:                              Sat, 26 Apr 2025   AIC                           1377.625
Time:                                      04:24:00   BIC                           1458.155
Sample:                                           0   HQIC                          1408.865
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.0966      0.165      0.584      0.559      -0.228       0.421
ma.L2          0.0236      0.149      0.158      0.874      -0.268       0.315
ma.L3          0.0013      0.155      0.009      0.993      -0.301       0.304
ma.L4         -0.0646      0.157     -0.410      0.682      -0.373       0.244
ma.L5         -0.1284      0.147     -0.872      0.383      -0.417       0.160
ma.L6         -0.1476      0.171     -0.864      0.388      -0.483       0.187
ma.L7         -0.1070      0.145     -0.738      0.460      -0.391       0.177
ma.L8         -0.0404      0.157     -0.257      0.797      -0.349       0.268
ma.L9          0.0075      0.150      0.050      0.960      -0.286       0.301
ma.L10         0.0666      0.149      0.447      0.655      -0.226       0.359
ma.L11         0.1376      0.164      0.837      0.403      -0.185       0.460
ma.L12        -0.8269      0.159     -5.216      0.000      -1.138      -0.516
ar.S.L12       0.0735      0.042      1.763      0.078      -0.008       0.155
ar.S.L24       0.0030      0.040      0.074      0.941      -0.076       0.082
ar.S.L36       0.0853      0.039      2.161      0.031       0.008       0.163
ar.S.L48      -0.0024      0.044     -0.055      0.956      -0.088       0.084
ar.S.L60       0.0263      0.035      0.749      0.454      -0.043       0.095
sigma2         0.4488      0.079      5.702      0.000       0.295       0.603
===================================================================================
Ljung-Box (L1) (Q):                   0.81   Jarque-Bera (JB):               306.79
Prob(Q):                              0.37   Prob(JB):                         0.00
Heteroskedasticity (H):               0.83   Skew:                             0.78
Prob(H) (two-sided):                  0.16   Kurtosis:                         5.99
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

Empeoró el AIC, entonces reducimos los valores de la parte AR.

## **26 MODELO**

Se busca mejorar el modelo 14 cambiando en una unidad las variables que se utilizarán.

In [193]:
modelo=SARIMAX(zpre,
               order=(0,0,12),
               seasonal_order=(3,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(0, 0, 12)x(3, 1, [], 12)   Log Likelihood                -671.005
Date:                              Sat, 26 Apr 2025   AIC                           1374.011
Time:                                      04:39:40   BIC                           1445.593
Sample:                                           0   HQIC                          1401.779
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.0979      0.147      0.668      0.504      -0.189       0.385
ma.L2          0.0290      0.133      0.218      0.827      -0.231       0.289
ma.L3         -0.0009      0.138     -0.007      0.995      -0.271       0.269
ma.L4         -0.0694      0.144     -0.483      0.629      -0.351       0.212
ma.L5         -0.1296      0.135     -0.960      0.337      -0.394       0.135
ma.L6         -0.1487      0.160     -0.930      0.352      -0.462       0.165
ma.L7         -0.1090      0.134     -0.816      0.414      -0.371       0.153
ma.L8         -0.0437      0.142     -0.309      0.758      -0.321       0.234
ma.L9          0.0059      0.134      0.044      0.965      -0.256       0.268
ma.L10         0.0726      0.133      0.546      0.585      -0.188       0.333
ma.L11         0.1389      0.147      0.944      0.345      -0.149       0.427
ma.L12        -0.8332      0.137     -6.068      0.000      -1.102      -0.564
ar.S.L12       0.0741      0.037      1.992      0.046       0.001       0.147
ar.S.L24       0.0121      0.037      0.324      0.746      -0.061       0.085
ar.S.L36       0.0879      0.037      2.349      0.019       0.015       0.161
sigma2         0.4456      0.070      6.380      0.000       0.309       0.582
===================================================================================
Ljung-Box (L1) (Q):                   0.84   Jarque-Bera (JB):               308.92
Prob(Q):                              0.36   Prob(JB):                         0.00
Heteroskedasticity (H):               0.83   Skew:                             0.80
Prob(H) (two-sided):                  0.16   Kurtosis:                         5.98
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

## **27 MODELO**

Como el modelo pasado mejoró, se reducen en una unidad las variables AR.

In [194]:
modelo=SARIMAX(zpre,
               order=(0,0,12),
               seasonal_order=(2,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(0, 0, 12)x(2, 1, [], 12)   Log Likelihood                -671.683
Date:                              Sat, 26 Apr 2025   AIC                           1373.366
Time:                                      04:40:20   BIC                           1440.474
Sample:                                           0   HQIC                          1399.399
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1132      0.033      3.473      0.001       0.049       0.177
ma.L2          0.0092      0.033      0.282      0.778      -0.055       0.073
ma.L3          0.0170      0.034      0.501      0.616      -0.050       0.084
ma.L4         -0.0621      0.041     -1.498      0.134      -0.143       0.019
ma.L5         -0.1440      0.040     -3.616      0.000      -0.222      -0.066
ma.L6         -0.1462      0.041     -3.598      0.000      -0.226      -0.067
ma.L7         -0.1187      0.038     -3.124      0.002      -0.193      -0.044
ma.L8         -0.0489      0.037     -1.320      0.187      -0.121       0.024
ma.L9          0.0127      0.035      0.359      0.720      -0.056       0.082
ma.L10         0.0613      0.037      1.665      0.096      -0.011       0.133
ma.L11         0.1594      0.037      4.354      0.000       0.088       0.231
ma.L12        -0.7519      0.043    -17.299      0.000      -0.837      -0.667
ar.S.L12       0.0097      0.038      0.257      0.797      -0.065       0.084
ar.S.L24      -0.0384      0.037     -1.033      0.301      -0.111       0.034
sigma2         0.4526      0.022     21.025      0.000       0.410       0.495
===================================================================================
Ljung-Box (L1) (Q):                   0.42   Jarque-Bera (JB):               336.43
Prob(Q):                              0.52   Prob(JB):                         0.00
Heteroskedasticity (H):               0.83   Skew:                             0.79
Prob(H) (two-sided):                  0.17   Kurtosis:                         6.16
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

## **28 MODELO**

Como el modelo pasado mejoró, se reducen en una unidad las variables AR.

In [195]:
modelo=SARIMAX(zpre,
               order=(0,0,12),
               seasonal_order=(1,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(0, 0, 12)x(1, 1, [], 12)   Log Likelihood                -672.395
Date:                              Sat, 26 Apr 2025   AIC                           1372.790
Time:                                      04:40:45   BIC                           1435.425
Sample:                                           0   HQIC                          1397.088
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1008      0.035      2.847      0.004       0.031       0.170
ma.L2          0.0093      0.035      0.265      0.791      -0.059       0.078
ma.L3          0.0095      0.036      0.265      0.791      -0.061       0.080
ma.L4         -0.0586      0.044     -1.348      0.178      -0.144       0.027
ma.L5         -0.1385      0.042     -3.301      0.001      -0.221      -0.056
ma.L6         -0.1535      0.046     -3.337      0.001      -0.244      -0.063
ma.L7         -0.1087      0.041     -2.678      0.007      -0.188      -0.029
ma.L8         -0.0352      0.038     -0.932      0.351      -0.109       0.039
ma.L9          0.0122      0.038      0.322      0.747      -0.062       0.086
ma.L10         0.0621      0.039      1.610      0.107      -0.013       0.138
ma.L11         0.1457      0.039      3.759      0.000       0.070       0.222
ma.L12        -0.7855      0.043    -18.232      0.000      -0.870      -0.701
ar.S.L12       0.0376      0.036      1.037      0.300      -0.034       0.109
sigma2         0.4511      0.022     20.259      0.000       0.407       0.495
===================================================================================
Ljung-Box (L1) (Q):                   0.88   Jarque-Bera (JB):               337.10
Prob(Q):                              0.35   Prob(JB):                         0.00
Heteroskedasticity (H):               0.83   Skew:                             0.81
Prob(H) (two-sided):                  0.17   Kurtosis:                         6.14
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

## **29 MODELO**

Como el modelo pasado mejoró, se reducen en una unidad las variables AR.

In [196]:
modelo=SARIMAX(zpre,
               order=(0,0,12),
               seasonal_order=(0,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(0, 0, 12)x(0, 1, [], 12)   Log Likelihood                -672.462
Date:                              Sat, 26 Apr 2025   AIC                           1370.924
Time:                                      04:41:05   BIC                           1429.085
Sample:                                           0   HQIC                          1393.487
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1044      0.033      3.189      0.001       0.040       0.169
ma.L2          0.0043      0.033      0.132      0.895      -0.060       0.068
ma.L3          0.0231      0.034      0.681      0.496      -0.043       0.089
ma.L4         -0.0506      0.041     -1.219      0.223      -0.132       0.031
ma.L5         -0.1395      0.040     -3.524      0.000      -0.217      -0.062
ma.L6         -0.1486      0.042     -3.559      0.000      -0.230      -0.067
ma.L7         -0.1134      0.038     -2.990      0.003      -0.188      -0.039
ma.L8         -0.0414      0.036     -1.138      0.255      -0.113       0.030
ma.L9          0.0121      0.036      0.339      0.735      -0.058       0.082
ma.L10         0.0523      0.037      1.415      0.157      -0.020       0.125
ma.L11         0.1440      0.036      4.047      0.000       0.074       0.214
ma.L12        -0.7644      0.036    -21.089      0.000      -0.835      -0.693
sigma2         0.4545      0.022     21.111      0.000       0.412       0.497
===================================================================================
Ljung-Box (L1) (Q):                   0.68   Jarque-Bera (JB):               332.20
Prob(Q):                              0.41   Prob(JB):                         0.00
Heteroskedasticity (H):               0.83   Skew:                             0.80
Prob(H) (two-sided):                  0.16   Kurtosis:                         6.12
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

## **30 MODELO**

Como no se pueden disminuir más las variables en la parte AR, se reduce la parte MA.

In [197]:
modelo=SARIMAX(zpre,
               order=(0,0,11),
               seasonal_order=(0,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(0, 0, 11)x(0, 1, [], 12)   Log Likelihood                -787.789
Date:                              Sat, 26 Apr 2025   AIC                           1599.578
Time:                                      04:41:23   BIC                           1653.265
Sample:                                           0   HQIC                          1620.405
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.4255      0.032     13.169      0.000       0.362       0.489
ma.L2          0.2088      0.035      6.025      0.000       0.141       0.277
ma.L3          0.2734      0.040      6.918      0.000       0.196       0.351
ma.L4         -0.0771      0.044     -1.760      0.078      -0.163       0.009
ma.L5         -0.2550      0.042     -6.073      0.000      -0.337      -0.173
ma.L6         -0.3467      0.044     -7.862      0.000      -0.433      -0.260
ma.L7         -0.2094      0.044     -4.800      0.000      -0.295      -0.124
ma.L8         -0.0935      0.043     -2.200      0.028      -0.177      -0.010
ma.L9          0.2441      0.044      5.576      0.000       0.158       0.330
ma.L10         0.2722      0.040      6.838      0.000       0.194       0.350
ma.L11         0.4646      0.037     12.677      0.000       0.393       0.536
sigma2         0.6580      0.029     22.499      0.000       0.601       0.715
===================================================================================
Ljung-Box (L1) (Q):                  21.40   Jarque-Bera (JB):               108.94
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.80   Skew:                             0.14
Prob(H) (two-sided):                  0.10   Kurtosis:                         4.99
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

El modelo empeoró.

## **31 MODELO**

Al reducir la parte MA empeoró el AIC, entonces se aumentará.

In [199]:
modelo=SARIMAX(zpre,
               order=(0,0,13),
               seasonal_order=(0,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(0, 0, 13)x(0, 1, [], 12)   Log Likelihood                -672.388
Date:                              Sat, 26 Apr 2025   AIC                           1372.777
Time:                                      04:42:01   BIC                           1435.411
Sample:                                           0   HQIC                          1397.075
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1321      0.039      3.423      0.001       0.056       0.208
ma.L2         -0.0049      0.034     -0.147      0.883      -0.071       0.061
ma.L3          0.0156      0.034      0.465      0.642      -0.050       0.082
ma.L4         -0.0566      0.039     -1.454      0.146      -0.133       0.020
ma.L5         -0.1449      0.037     -3.917      0.000      -0.217      -0.072
ma.L6         -0.1510      0.043     -3.546      0.000      -0.235      -0.068
ma.L7         -0.1027      0.039     -2.654      0.008      -0.179      -0.027
ma.L8         -0.0273      0.036     -0.757      0.449      -0.098       0.043
ma.L9          0.0229      0.036      0.643      0.520      -0.047       0.093
ma.L10         0.0574      0.034      1.666      0.096      -0.010       0.125
ma.L11         0.1408      0.034      4.182      0.000       0.075       0.207
ma.L12        -0.7775      0.034    -22.609      0.000      -0.845      -0.710
ma.L13        -0.0451      0.040     -1.124      0.261      -0.124       0.034
sigma2         0.4518      0.021     21.541      0.000       0.411       0.493
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               318.78
Prob(Q):                              0.96   Prob(JB):                         0.00
Heteroskedasticity (H):               0.85   Skew:                             0.80
Prob(H) (two-sided):                  0.24   Kurtosis:                         6.04
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

En base al AIC, el modelo empeoró.

## **32 MODELO**

Como el modelo con mayor AIC era el que tenia 12 variables en la parte MA, se propone que mejor sea una en la parte MA estacional.

In [200]:
modelo=SARIMAX(zpre,
               order=(0,0,0),
               seasonal_order=(0,1,1,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 SARIMAX Results                                  
==================================================================================
Dep. Variable:                          y   No. Observations:                  660
Model:             SARIMAX(0, 1, [1], 12)   Log Likelihood                -719.797
Date:                    Sat, 26 Apr 2025   AIC                           1443.593
Time:                            04:42:19   BIC                           1452.541
Sample:                                 0   HQIC                          1447.064
                                    - 660                                         
Covariance Type:                      opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.S.L12      -0.6533      0.025    -26.057      0.000      -0.702      -0.604
sigma2         0.5344      0.019     28.147      0.000       0.497       0.572
===================================================================================
Ljung-Box (L1) (Q):                  15.79   Jarque-Bera (JB):               286.71
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.83   Skew:                             0.59
Prob(H) (two-sided):                  0.17   Kurtosis:                         6.04
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

En base al AIC, el modelo empeoró.

## **33 MODELO**

Se combinan las partes estacionales y no estacionales.

In [202]:
modelo=SARIMAX(zpre,
               order=(0,0,11),
               seasonal_order=(0,1,1,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                       SARIMAX Results                                       
=============================================================================================
Dep. Variable:                                     y   No. Observations:                  660
Model:             SARIMAX(0, 0, 11)x(0, 1, [1], 12)   Log Likelihood                -686.981
Date:                               Sat, 26 Apr 2025   AIC                           1399.963
Time:                                       04:49:23   BIC                           1458.123
Sample:                                            0   HQIC                          1422.525
                                               - 660                                         
Covariance Type:                                 opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1367      0.033      4.088      0.000       0.071       0.202
ma.L2          0.0373      0.037      0.998      0.318      -0.036       0.111
ma.L3          0.0723      0.038      1.886      0.059      -0.003       0.147
ma.L4         -0.0787      0.047     -1.677      0.094      -0.171       0.013
ma.L5         -0.1678      0.054     -3.086      0.002      -0.274      -0.061
ma.L6         -0.1570      0.061     -2.558      0.011      -0.277      -0.037
ma.L7         -0.0491      0.050     -0.991      0.322      -0.146       0.048
ma.L8         -0.0516      0.044     -1.162      0.245      -0.139       0.035
ma.L9          0.0499      0.043      1.161      0.246      -0.034       0.134
ma.L10         0.0851      0.043      1.975      0.048       0.001       0.170
ma.L11         0.1262      0.040      3.160      0.002       0.048       0.205
ma.S.L12      -0.7353      0.024    -30.318      0.000      -0.783      -0.688
sigma2         0.4801      0.020     23.810      0.000       0.441       0.520
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               242.28
Prob(Q):                              0.98   Prob(JB):                         0.00
Heteroskedasticity (H):               0.83   Skew:                             0.68
Prob(H) (two-sided):                  0.18   Kurtosis:                         5.67
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

En base al AIC, el modelo empeoró.

## **MODELO 29**

Se concluye que, basado en el AIC, el mejor modelo es el 29

## **MODELO 34**

In [6]:
modelo=SARIMAX(zpre,
               order=(0,0,15),
               seasonal_order=(0,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(0, 0, 15)x(0, 1, [], 12)   Log Likelihood                -669.403
Date:                              Sat, 26 Apr 2025   AIC                           1370.806
Time:                                      19:41:19   BIC                           1442.388
Sample:                                           0   HQIC                          1398.575
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1367      0.037      3.666      0.000       0.064       0.210
ma.L2          0.0006      0.041      0.014      0.988      -0.080       0.081
ma.L3          0.0767      0.041      1.866      0.062      -0.004       0.157
ma.L4         -0.0764      0.040     -1.892      0.058      -0.155       0.003
ma.L5         -0.1454      0.038     -3.816      0.000      -0.220      -0.071
ma.L6         -0.1591      0.041     -3.922      0.000      -0.239      -0.080
ma.L7         -0.0977      0.038     -2.566      0.010      -0.172      -0.023
ma.L8         -0.0202      0.036     -0.554      0.580      -0.091       0.051
ma.L9          0.0189      0.035      0.543      0.587      -0.049       0.087
ma.L10         0.0721      0.034      2.104      0.035       0.005       0.139
ma.L11         0.1653      0.035      4.718      0.000       0.097       0.234
ma.L12        -0.7787      0.036    -21.409      0.000      -0.850      -0.707
ma.L13        -0.0483      0.041     -1.178      0.239      -0.129       0.032
ma.L14        -0.0184      0.039     -0.469      0.639      -0.095       0.059
ma.L15        -0.1024      0.043     -2.372      0.018      -0.187      -0.018
sigma2         0.4461      0.021     21.164      0.000       0.405       0.487
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):               360.03
Prob(Q):                              0.94   Prob(JB):                         0.00
Heteroskedasticity (H):               0.80   Skew:                             0.86
Prob(H) (two-sided):                  0.10   Kurtosis:                         6.22
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [8]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p1': 0.13670240371999162, 'p2': 0.0005971888480305956, 'p3': 0.07670095541918637, 'p4': -0.07637154759112381, 'p5': -0.14537162831990777, 'p6': -0.1591494261114962, 'p7': -0.09765166095390075, 'p8': -0.020150469201547817, 'p9': 0.018894086400221687, 'p10': 0.07211092136133603, 'p11': 0.1652919906997901, 'p12': -0.7787367794635099, 'p13': -0.04830339922205925, 'p14': -0.018407433802944403, 'p15': -0.10242070791381097}

El grado del polinomio es: 15

Raíces del polinomio característico: [ 0.8140251 +1.79974151j  0.8140251 -1.79974151j -2.00260148+0.j
  0.95007946+0.50126699j  0.95007946-0.50126699j  1.00203779+0.j
  0.51057431+0.86905551j  0.51057431-0.86905551j  0.00842192+1.0215353j
  0.00842192-1.0215353j  -0.50272798+0.87137081j -0.50272798-0.87137081j
 -1.00175188+0.j         -0.8690769 +0.50012473j -0.8690769 -0.50012473j]

Módulo de las raíces: [1.97527375 1.97527375 2.00260148 1.07420649 1.07420649 1.00203779
 1.00794028 1.00794028 1.021

In [9]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.269005,NaN
2,0.273263,NaN
3,0.399129,NaN
4,0.417800,NaN
5,1.551264,NaN
6,1.980990,NaN
7,2.978212,NaN
8,3.536694,NaN
9,4.544775,NaN
10,4.576955,NaN


Todos los residuos son independientes.

In [10]:
modelo.resid.mean()

-0.045524455453858546

Los residuos son un poco mejores que el modelo 14.

In [11]:
ttest_1samp(modelo.resid, 0)

TtestResult(statistic=-1.7052893997409186, pvalue=0.08861172351622751, df=659)

La media de los residuos es cero.

In [12]:
het_breuschpagan(modelo.resid, add_constant(np.arange(len(modelo.resid))))

(2.246413161692278, 0.1339245098780731, 2.247254731819336, 0.13433127866106115)

Los residuos son constantes.

In [13]:
print(jarque_bera(modelo.resid))
print(lilliefors(modelo.resid))

SignificanceResult(statistic=399.6392527449315, pvalue=1.6574441016528617e-87)
(0.11286846354370039, 0.0009999999999998899)


No se distribuye normal.

In [14]:
normal_relajacion(modelo.resid)

74.55% de los residuos están dentro de ±1σ (esperado ≈ 68%)
94.39% de los residuos están dentro de ±2σ (esperado ≈ 95%)
98.64% de los residuos están dentro de ±3σ (esperado ≈ 99.7%)


## **MODELO 35**

In [15]:
modelo=SARIMAX(zpre,
               order=(3,0,12),
               seasonal_order=(0,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(3, 0, 12)x(0, 1, [], 12)   Log Likelihood                -670.126
Date:                              Sat, 26 Apr 2025   AIC                           1372.252
Time:                                      19:48:34   BIC                           1443.835
Sample:                                           0   HQIC                          1400.021
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0899      0.054      1.661      0.097      -0.016       0.196
ar.L2          0.0370      0.051      0.719      0.472      -0.064       0.138
ar.L3          0.1178      0.055      2.131      0.033       0.009       0.226
ma.L1          0.0597      0.044      1.357      0.175      -0.027       0.146
ma.L2         -0.0522      0.046     -1.135      0.256      -0.142       0.038
ma.L3         -0.0470      0.046     -1.030      0.303      -0.136       0.042
ma.L4         -0.0852      0.040     -2.142      0.032      -0.163      -0.007
ma.L5         -0.1400      0.037     -3.812      0.000      -0.212      -0.068
ma.L6         -0.1454      0.040     -3.661      0.000      -0.223      -0.068
ma.L7         -0.0729      0.043     -1.693      0.090      -0.157       0.011
ma.L8          0.0061      0.042      0.146      0.884      -0.076       0.089
ma.L9          0.0427      0.040      1.069      0.285      -0.036       0.121
ma.L10         0.0887      0.035      2.547      0.011       0.020       0.157
ma.L11         0.1608      0.033      4.904      0.000       0.097       0.225
ma.L12        -0.7827      0.037    -21.274      0.000      -0.855      -0.711
sigma2         0.4532      0.021     21.714      0.000       0.412       0.494
===================================================================================
Ljung-Box (L1) (Q):                   0.19   Jarque-Bera (JB):               359.23
Prob(Q):                              0.66   Prob(JB):                         0.00
Heteroskedasticity (H):               0.80   Skew:                             0.84
Prob(H) (two-sided):                  0.11   Kurtosis:                         6.24
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [16]:
estacionario(modelo)

El polinomio de la parte autoregresiva es: {'p1': -0.08991417437089785, 'p2': -0.03698974923423608, 'p3': -0.11775049355480943}

El grado del polinomio es: 3

Raíces del polinomio característico: [-1.06848768+1.87544516j -1.06848768-1.87544516j  1.82283868+0.j        ]

Módulo de las raíces: [2.15846253 2.15846253 1.82283868]

¿Las raíces están fuera del círculo unitario?  True

El modelo es estacionario

:)


In [17]:
invertible(modelo)

El polinomio de la parte media móvil es: {'p1': 0.05970879349005376, 'p2': -0.05218473953261615, 'p3': -0.04697728845666392, 'p4': -0.08516376860814465, 'p5': -0.14002894727331172, 'p6': -0.14543294963636855, 'p7': -0.07292322916953548, 'p8': 0.006138862312847715, 'p9': 0.04270502381902491, 'p10': 0.08870873089915969, 'p11': 0.1607553325085041, 'p12': -0.7827250554485619}

El grado del polinomio es: 12

Raíces del polinomio característico: [ 1.00359241+0.j          0.95752189+0.50127658j  0.95752189-0.50127658j
  0.51075225+0.86925369j  0.51075225-0.86925369j  0.00912594+1.02183589j
  0.00912594-1.02183589j -0.50441485+0.87243904j -0.50441485-0.87243904j
 -1.0024223 +0.j         -0.87088075+0.50008891j -0.87088075-0.50008891j]

Módulo de las raíces: [1.00359241 1.08079896 1.08079896 1.00820129 1.00820129 1.02187664
 1.02187664 1.00776198 1.00776198 1.0024223  1.00425206 1.00425206]

¿Las raíces están fuera del círculo unitario?  True

El modelo es invertible

:)


In [18]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.023666,NaN
2,0.086887,NaN
3,0.198990,NaN
4,0.419499,NaN
5,1.560612,NaN
6,2.321916,NaN
7,3.614509,NaN
8,4.128976,NaN
9,5.259795,NaN
10,5.391771,NaN


Todos los residuos son independientes.

In [19]:
modelo.resid.mean()

-0.03916177394709288

Los residuos son un poco mejores que el modelo 14.

In [20]:
ttest_1samp(modelo.resid, 0)

TtestResult(statistic=-1.4650214155998478, pvalue=0.14339198238771567, df=659)

La media de los residuos es cero.

In [21]:
het_breuschpagan(modelo.resid, add_constant(np.arange(len(modelo.resid))))

(2.0446024101041016, 0.1527468863710772, 2.044741012501057, 0.1532076118131389)

Los residuos son constantes.

In [22]:
print(jarque_bera(modelo.resid))
print(lilliefors(modelo.resid))

SignificanceResult(statistic=393.74420679325874, pvalue=3.1588707333278383e-86)
(0.11789404576529727, 0.0009999999999998899)


No se distribuye normal.

In [23]:
normal_relajacion(modelo.resid)

74.70% de los residuos están dentro de ±1σ (esperado ≈ 68%)
94.24% de los residuos están dentro de ±2σ (esperado ≈ 95%)
98.79% de los residuos están dentro de ±3σ (esperado ≈ 99.7%)


# **MODELO PROPUESTO**

Se proponen modelos que cumplen los supuestos

29, 23

$$
\text{ARIMA}(0,0,12) \times (4,1,0)_{12} \text{ para la serie $X_t$}
$$

con los coeficientes:

- $\phi_1 = 0.1691785261672151$
- $\Theta_1 = 0.6868323986912898$

$$
(1
- \phi_1B)W_t=
(1
- \Theta_1 B^{12})\varepsilon_{t}
$$

Donde $Wt = \nabla_{12}^1 X_t = X_t - X_{t-12}$ y $\varepsilon_t \sim \mathcal{N}(0, 1)$

In [ ]:
modelo=SARIMAX(zpre,order=(0,0,12),seasonal_order=(4,1,0,12)).fit().summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(0, 0, 12)x(4, 1, [], 12)   Log Likelihood                -670.963
Date:                              Sat, 26 Apr 2025   AIC                           1375.925
Time:                                      04:20:33   BIC                           1451.982
Sample:                                           0   HQIC                          1405.430
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.0982      0.150      0.655      0.513      -0.196       0.392
ma.L2          0.0265      0.134      0.197      0.844      -0.237       0.290
ma.L3      -4.313e-05      0.139     -0.000      1.000      -0.273       0.273
ma.L4         -0.0667      0.144     -0.463      0.643      -0.349       0.216
ma.L5         -0.1299      0.135     -0.961      0.337      -0.395       0.135
ma.L6         -0.1508      0.159     -0.948      0.343      -0.462       0.161
ma.L7         -0.1085      0.133     -0.815      0.415      -0.369       0.152
ma.L8         -0.0413      0.143     -0.290      0.772      -0.321       0.238
ma.L9          0.0055      0.136      0.040      0.968      -0.260       0.271
ma.L10         0.0699      0.135      0.518      0.605      -0.195       0.335
ma.L11         0.1394      0.151      0.925      0.355      -0.156       0.435
ma.L12        -0.8294      0.143     -5.802      0.000      -1.110      -0.549
ar.S.L12       0.0730      0.039      1.885      0.059      -0.003       0.149
ar.S.L24       0.0068      0.039      0.174      0.862      -0.069       0.083
ar.S.L36       0.0849      0.038      2.237      0.025       0.010       0.159
ar.S.L48       0.0030      0.043      0.069      0.945      -0.082       0.088
sigma2         0.4437      0.071      6.256      0.000       0.305       0.583
===================================================================================
Ljung-Box (L1) (Q):                   0.83   Jarque-Bera (JB):               310.22
Prob(Q):                              0.36   Prob(JB):                         0.00
Heteroskedasticity (H):               0.83   Skew:                             0.80
Prob(H) (two-sided):                  0.16   Kurtosis:                         5.99
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""